# Quick loop on twin color pairs  with multi colors Z-Y And Simulate Atm effect Version 3

- author Sylvie Dagoret-Campagne
- creation date 2024-09-08   : time_sep = 1800.
- last update : 2024-09-09  : choose time_sep = 3600. and show SED and data point. Use mapping file special Z-Y
- last update : 2024-09-12 : can read other version of results and use the improved model on colors
- last update : 2024-09-15 : cosmetic

- affiliation : IJCLab
- Kernel @usdf **w_2024_16**
- Office emac : mamba_py311
- Home emac : base (conda)
- laptop : conda_py310

** Run LoopTwinSource before this notebook**
Must do quick run


In addition this notebook does simulation. It needs:
- getObsAtmo
- rubinsimphot
  

In [ ]:
import warnings
warnings.resetwarnings()
warnings.simplefilter('ignore')

In [ ]:
from platform import python_version
print(python_version())

In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm,SymLogNorm
from matplotlib.patches import Circle,Annulus
from astropy.visualization import ZScaleInterval
props = dict(boxstyle='round', facecolor="white", alpha=0.1)
#props = dict(boxstyle='round')

import matplotlib.colors as colors
import matplotlib.cm as cmx

import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from matplotlib.gridspec import GridSpec

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.io import fits
from astropy.wcs import WCS
from astropy import units as u
from astropy import constants as c

from scipy import interpolate
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KDTree, BallTree

import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option('display.max_rows', 100)

import matplotlib.ticker                         # here's where the formatter is
import os
import re
import pandas as pd
import pickle
from collections import OrderedDict

plt.rcParams["figure.figsize"] = (4,3)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'

import scipy
from scipy.optimize import curve_fit

# new color correction model
import pickle
from scipy.interpolate import RegularGridInterpolator

In [ ]:
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.time import Time


### Import getObsAtmo

- to simulate atmosphere

In [ ]:
from importlib.metadata import version
the_ver = version('getObsAtmo')
print(f"Version of getObsAtmo : {the_ver}")

In [ ]:
from getObsAtmo import ObsAtmo
emul = ObsAtmo("AUXTEL")

In [ ]:
WL = emul.GetWL()

### import Library that encapsulate calculations for Photometric correction

from my package RubinLSSTPhotometricCorrTuto

In [ ]:
import sys
sys.path.append('../lib')
#import libAtmosphericFit

In [ ]:
# This package encapsulate the calculation on calibration used in this nb
from libPhotometricCorrections_auxtel import *
photom_wl = np.array([FILTERWL[1,2],FILTERWL[2,2],FILTERWL[3,2],FILTERWL[4,2],FILTERWL[5,2]])

In [ ]:
def set_photometric_parameters(exptime, nexp, readnoise=None):
    # readnoise = None will use the default (8.8 e/pixel). Readnoise should be in electrons/pixel.
    photParams = PhotometricParameters(exptime=exptime, nexp=nexp, readnoise=readnoise)
    return photParams

In [ ]:
def scale_sed(ref_mag, ref_filter, sed):
    fluxNorm = sed.calc_flux_norm(ref_mag, lsst_std[ref_filter])
    sed.multiply_flux_norm(fluxNorm)
    return sed


In [ ]:
def ComputeColors(the_pc,the_sed1,the_sed2):
    
    all_df_magcol = []
    for index,pwv in enumerate(all_pwv):
        atm_bands = the_pc.coll_bandpass_total_nonstd[index]
        mags_pwv1 = {}
        mags_pwv2 = {}
        madus_pwv1 = {}
        madus_pwv2 = {}
        for f in filter_tagnames: 
            mags_pwv1[f] = the_sed1.calc_mag(atm_bands[f])
            mags_pwv2[f] = the_sed2.calc_mag(atm_bands[f])
            madus_pwv1[f] = -2.5*np.log10(the_sed1.calc_adu(atm_bands[f],photoparams))
            madus_pwv2[f] = -2.5*np.log10(the_sed2.calc_adu(atm_bands[f],photoparams))
        # AB colors for star 1
        mags_pwv1["g-r"] = mags_pwv1["g"]- mags_pwv1["r"]  
        mags_pwv1["z-y"] = mags_pwv1["z"]- mags_pwv1["y"] 
        # AB colors for star 2
        mags_pwv2["g-r"] = mags_pwv2["g"]- mags_pwv2["r"]  
        mags_pwv2["z-y"] = mags_pwv2["z"]- mags_pwv2["y"] 
        # ADU mag colors for star 1
        # MADU colors for star 1
        madus_pwv1["g-r"] = madus_pwv1["g"]- madus_pwv1["r"]  
        madus_pwv1["z-y"] = madus_pwv1["z"]- madus_pwv1["y"] 
        # MADU colors for star 2
        madus_pwv2["g-r"] = madus_pwv2["g"]- madus_pwv2["r"]  
        madus_pwv2["z-y"] = madus_pwv2["z"]- madus_pwv2["y"] 
        
        df1 = pd.DataFrame(mags_pwv1, index=["abmag_1"]).T
        df2 = pd.DataFrame(mags_pwv2, index=["abmag_2"]).T
        df3 = pd.DataFrame(madus_pwv1, index=["instmag_1"]).T
        df4 = pd.DataFrame(madus_pwv2, index=["instmag_2"]).T
        df = pd.concat([df1,df2,df3,df4],axis=1)
        all_df_magcol.append(df)
        
    return all_df_magcol

In [ ]:
def ComputeRelativeDiffColor(the_all_df_magcol):

    df_diffcol = pd.DataFrame(columns=["pwv","Czy1_Czy2_abmag","Czy1_Czy2_instmag"])
    for index,pwv in enumerate(all_pwv):
        the_df = the_all_df_magcol[index]
        DCzyabmag = the_df.loc["z-y","abmag_1"] - the_df.loc["z-y","abmag_2"]
        DCzyadumag = the_df.loc["z-y","instmag_1"] - the_df.loc["z-y","instmag_2"]
        df_diffcol.loc[index] = [pwv,DCzyabmag,DCzyadumag]

    # Relative difference color is in mmag for plotting
    df_diffcol["Czy1_Czy2_abmag_rel"] =  (df_diffcol["Czy1_Czy2_abmag"] - df_diffcol.loc[0,"Czy1_Czy2_abmag"])*1000
    df_diffcol["Czy1_Czy2_instmag_rel"] =  (df_diffcol["Czy1_Czy2_instmag"] - df_diffcol.loc[0,"Czy1_Czy2_instmag"])*1000

    # differentiation  mmag per mm
    dx = np.diff(df_diffcol.pwv.values).mean()
    dy1 = np.diff(df_diffcol["Czy1_Czy2_abmag_rel"].values)
    dy2 = np.diff(df_diffcol["Czy1_Czy2_instmag_rel"].values)
    z_y0 = df_diffcol.loc[0,"Czy1_Czy2_abmag"]
    return df_diffcol

In [ ]:
def getSEDfromPickle(fn_sed,spectype,the_pc,refcalimag=20.0,reffilt="z"):
    """
    parameters:
      fn_sed : filename of pickle sed
      spectype : spectro type of the spectrum
      the_pc : PhotometricCorrections object
      refcalimag : magnitude to normalise the SED
      reffilt : filter to which apply the refcalimag
    """
    
    fullfilename = os.path.join(seddir_uvk,fn_sed) 
    hdul = fits.open(fullfilename)
    dff = pd.DataFrame(hdul[1].data)
    
    wl= dff.WAVELENGTH.values
    flux = dff.FLUX.values
    #wavelen (nm)
    
    #flambda (ergs/cm^2/s/nm)
    the_sed = Sed(wavelen=wl/10., flambda=flux*10., name=spectype)
    flux_norm = the_sed.calc_flux_norm(refcalimag, the_pc.bandpass_total_std[reffilt])
    the_sed.multiply_flux_norm(flux_norm)
    return the_sed
    

### import library rubin_sim defining LSST parameters, namely for photometric calculations including access to Pickles templates

In [ ]:
from rubinsimphot.phot_utils import Bandpass, Sed
from rubinsimphot.data import get_data_dir

### Config for the color correction emulator

In [ ]:
OBS_tag = "AUXTEL"
file0_colordiffmodel = f"ModelSEDColorCorrections/Z_Y/{OBS_tag}_atmosphericzycolorgrid_params.pickle"
file1_colordiffmodel = f"ModelSEDColorCorrections/Z_Y/{OBS_tag}_atmosphericzycolorgrid_datagrid.npy"

In [ ]:
with (open(file0_colordiffmodel, "rb")) as openfile:
    while True:
        try:
            info_params_colordiffmodel = pickle.load(openfile)
        except EOFError:
            break

In [ ]:
info_params_colordiffmodel

In [ ]:
all_pwv_colordiffmodel = info_params_colordiffmodel['PWV']
all_zycolordiff_colordiffmodel = info_params_colordiffmodel['ZYCOLDIFF']

In [ ]:
datagrid_colordiffmodel = np.load(file1_colordiffmodel,mmap_mode=None, allow_pickle=False)
print(f"...... datagrid file for colordiffmodel{file1_colordiffmodel} read")

## Start Here

In [ ]:
# reference flux in Jy
F0 = ((0.*u.ABmag).to(u.Jy)).value
F0

#### basic test for libPhotometricCorrections_auxtel

In [ ]:
am0 = 1.20    # airmass
pwv0 = 3.0  # Precipitable water vapor vertical column depth in mm
oz0 = 300.  # Ozone vertical column depth in Dobson Unit (DU)
ncomp = 1     # Number of aerosol components
tau0 = 0.04 # Vertical Aerosol depth (VAOD) 
beta0 = 1.2 # Aerosol Angstrom exponent

In [ ]:
#same reference 

pc = PhotometricCorrections(am0,pwv0,oz0,tau0,beta0)
pc1 = PhotometricCorrections(am0,pwv0,oz0,tau0,beta0)
pc2 = PhotometricCorrections(am0,pwv0,oz0,tau0,beta0)

In [ ]:
am = 1.0
oz = 300
tau= 0.04
beta = 1.2
all_pwv = np.arange(0,15.,0.5)
NPWV = len(all_pwv)

In [ ]:
pc.CalculateMultiObs(am,all_pwv,oz,tau,beta)
pc1.CalculateMultiObs(1.2,all_pwv,oz,tau,beta)
pc2.CalculateMultiObs(2.,all_pwv,oz,tau,beta)

In [ ]:
NOBS = len(all_pwv)

# wavelength bin colors
jet = plt.get_cmap('jet')
cNorm = colors.Normalize(vmin=0, vmax=NOBS)
scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)
all_colors = scalarMap.to_rgba(np.arange(NOBS), alpha=1)


fig,axs = plt.subplots(1,1,figsize=(6,4))

# Figure 1
#axs=fig.add_subplot(1,1,1)
for index,pwv in enumerate(all_pwv):
  
    atm_bands = pc.coll_bandpass_total_nonstd[index]

    label = f"pwv={pwv:.1f}" 

    for f in filter_tagnames: 
        axs.plot(atm_bands[f].wavelen,atm_bands[f].sb,color=all_colors[index],label=label,lw=0.5)
 
   
axs.plot(pc.WL,pc.atm_std,color="k",lw=2,label="standard atmosphere")
#axs.legend(bbox_to_anchor=(1.03, 1.0))  
axs.set_xlabel("$\lambda$ (nm)")
axs.set_title("standard and observed transmission for airmass 1")

ax2 = axs.twinx()
for ifilt,f in enumerate(filter_tagnames):
    ax2.fill_between(pc.bandpass_total_std[f].wavelen,pc.bandpass_total_std[f].sb,color=filter_color[ifilt],alpha=0.1) 
    ax2.set_yticks([])


plt.tight_layout()
plt.show()


#### Check SED Pickles

In [ ]:
# Find the throughputs directory 
#fdir = os.getenv('RUBIN_SIM_DATA_DIR')
fdir = get_data_dir()
if fdir is None:  #environment variable not set
    fdir = os.path.join(os.getenv('HOME'), 'rubin_sim_data')

In [ ]:
seddir = os.path.join(fdir, 'pysynphot', 'pickles')
seddir_uvk = os.path.join(seddir,"dat_uvk")
all_pickles_uvk = sorted(os.listdir(seddir_uvk))
file_ref = os.path.join(seddir_uvk, "pickles_uk.fits")
hdul = fits.open(file_ref)
df_pickle = pd.DataFrame(hdul[1].data)
print(df_pickle.head())
NSED = len(df_pickle)

In [ ]:
# wavelength bin colors
jet = plt.get_cmap('jet')
cNorm = mpl.colors.Normalize(vmin=0, vmax=NSED)
scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)
all_colors = scalarMap.to_rgba(np.arange(NSED), alpha=1)

fig, ax = plt.subplots(figsize=(18, 0.2), layout='constrained')
cmap = mpl.cm.jet
norm = mpl.colors.Normalize(vmin=0, vmax=NSED)
cbar = fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),cax=ax, orientation='horizontal', label='spectral type')

# change the number of labels
labels = [item.get_text() for item in cbar.ax.get_xticklabels()]
cbar.ax.locator_params(axis='x', nbins=4*len(labels)) 
labels = [item.get_text() for item in cbar.ax.get_xticklabels()]

# rename the labels
Nlabels = len(labels)
for ilab in range(0,Nlabels-1):
    the_label = int(labels[ilab])
    labels[ilab] = df_pickle.loc[the_label,"SPTYPE"]
cbar.ax.set_xticklabels(labels,rotation=45);
plt.show()


fig,ax = plt.subplots(1,1,figsize=(8,6))
for index in np.arange(NSED):
    filename = df_pickle.loc[index,"FILENAME"].strip()+".fits"
    fullfilename = os.path.join(seddir_uvk,filename) 
    hdul = fits.open(fullfilename)
    dff = pd.DataFrame(hdul[1].data)
    if index < 100 or index > 104: 
        ax.plot(dff.WAVELENGTH,dff.FLUX,color=all_colors[index])
ax.set_yscale('log')
ax.set_xlim(3000.,11000)
ax.set_ylim(1e-11,1e-6)
ax.grid()
ax.set_title("Pickles $F_\lambda$")
ax.set_ylabel("$F_\lambda$")
ax.set_xlabel("$\lambda \, (\AA$)")
#plt.colorbar(ax.collections[0],ax=ax,orientation = 'horizontal',shrink=0.8)
#ax.collections[-1].colorbar

cbar =fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),ax=ax, orientation='vertical', label='spectral type')

#cbar = plt.colorbar(ax.collections[0],ax=ax)
#cbar.ax.set_ylabel('spectral type', rotation=90)

labels = [item.get_text() for item in cbar.ax.get_yticklabels()]
cbar.ax.locator_params(axis='y', nbins=2*len(labels)) 
labels = [item.get_text() for item in cbar.ax.get_yticklabels()]

Nlabels = len(labels)
for ilab in range(0,Nlabels-1):
    the_label = int(labels[ilab])
    labels[ilab] = df_pickle.loc[the_label,"SPTYPE"]
cbar.ax.set_yticklabels(labels,rotation=0);
plt.tight_layout()


In [ ]:
fig, ax = plt.subplots(figsize=(18, 0.6), layout='constrained')
cmap = mpl.cm.jet
norm = mpl.colors.Normalize(vmin=0, vmax=NSED)
cbar = fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),cax=ax, orientation='horizontal', label='spectral type')

# change the number of labels
labels = [item.get_text() for item in cbar.ax.get_xticklabels()]
cbar.ax.locator_params(axis='x', nbins=4*len(labels)) 
labels = [item.get_text() for item in cbar.ax.get_xticklabels()]

# rename the labels
Nlabels = len(labels)
for ilab in range(0,Nlabels-1):
    the_label = int(labels[ilab])
    labels[ilab] = df_pickle.loc[the_label,"SPTYPE"]
cbar.ax.set_xticklabels(labels,rotation=45);
plt.show()

In [ ]:
transform = AsinhStretch() + PercentileInterval(99.)

In [ ]:
# INSERT YOUR collection and tract
# for rehearsal use collection 2 which have CCDvisit
butlerRoot = "/repo/embargo"

collection1 = 'LSSTComCamSim/runs/nightlyvalidation/20240402/d_2024_03_29/DM-43612'
collection2 = 'LSSTComCamSim/runs/nightlyvalidation/20240403/d_2024_03_29/DM-43612'
collection3 = 'LSSTComCamSim/runs/nightlyvalidation/20240404/d_2024_03_29/DM-43612'
collection = 'LATISS/runs/AUXTEL_DRP_IMAGING_20230509_20240414/w_2024_15/PREOPS-5069' # COMPLETED
collection = 'LATISS/runs/AUXTEL_DRP_IMAGING_20230509_20240513/w_2024_20/PREOPS-5146' # test this one today 2024/06/22

collectionn = collection
#collections = [collection1,collection2,collection3]
collections = [collection]
collectionStr = collectionn.replace("/", "_")
fn_ccdVisit_tracts_patches = f"ccdVisittractpatch_{collectionStr}.csv"
instrument = 'LATISS'
skymapName = "latiss_v1"
where_clause = "instrument = \'" + instrument+ "\'"
tract = 3864 # mostly for light-curves
patch_sel = 236
#tract = 5615
# tract = 5634 # interesting to view calib parameters
suptitle = collectionStr + f" inst = {instrument} tract = {tract}"

In [ ]:
# Max angular distance betwwen stars of the pair
sep_max = 300.
sep_time = 7200.
version_results = "v2"

## Definition of functions used

In [ ]:
def convert_fluxtomag(x) :
    """
    The object and source catalogs store only fluxes. There are hundreds of flux-related columns, 
    and to store them also as magnitudes would be redundant, and a waste of space.
    All flux units are nanojanskys. The AB Magnitudes Wikipedia page provides a concise resource 
    for users unfamiliar with AB magnitudes and jansky fluxes. To convert to AB magnitudes use:
    As demonstrated in Section 2.3.2, to add columns of magnitudes after retrieving columns of flux, users can do this:
    results_table['r_calibMag'] = -2.50 * numpy.log10(results_table['r_calibFlux']) + 31.4
    results_table['r_cModelMag'] = -2.50 * numpy.log10(results_table['r_cModelFlux']) + 31.4
    (from DP0 tutorial)
    """
    return -2.50 * np.log10(x) + 31.4

In [ ]:
def closestvisit(df1,df2):
    """
    This function associate pairs of closest observation for photmetry

    input : the 2 dataframes (first to one for which one want to find the closest obs in dtaframe 2)
    output : the dataframe of similar format as the first one having columns on nightobs , visit time, dt from second dataframe)
    """
    d1 =df1.copy(deep=True)
    d2 =df2.copy(deep=True)
    all_samples = []
    df = pd.DataFrame(index=d1.index,columns=["nightObs2","visit2","time2","deltat"])
    for key in d1.index:

        nightObs = key[0]
        time1 = d1.loc[key]["time"]
        d2["dt"] = np.abs(d2["time"]-time1)
        d2["Dt"] = d2["time"]-time1
        cut_sel = (d2["dt"] == d2["dt"].min()) #& (d2["nightObs"] == nightObs)
        sample = d2[cut_sel]
        #df.iloc[key] = [sample.index[0][0],sample.index[0][1],sample["time"].values[0],sample['dt'].values[0]]
        df.loc[key]["nightObs2"] = sample.index[0][0]
        df.loc[key]["visit2"] = sample.index[0][1]
        df.loc[key]["time2"] = sample["time"].values[0]
        df.loc[key]["deltat"] = pd.to_timedelta(sample["Dt"].values[0], unit="s").total_seconds()
        all_samples.append(sample)

    # select observations during the same night
    df_back = df.copy(deep=True)
    flag_sel = []
    for key in df_back.index:
        nightObs = key[0]
        nightObs2 = df_back.loc[key]["nightObs2"]
        flag_sel.append(nightObs == nightObs2)
    df = df[flag_sel]
    return df

In [ ]:
def getcutoutandimage(visitId,df_selectedvisits):
    """
    visitId : the calexp where are the two stars
    df_selectedvisits : the list of all sources and visit of the main targeted star
    """

    row_source = df_selectedvisits[df_selectedvisits.visit == visitId].iloc[0]  
    band = row_source['band']

    dataId = {'visit': visitId, 'instrument':instrument , 'detector': 0}
    calexp = butler.get('calexp', **dataId,collections=collections)   
    src_cat = butler.get('sourceTable',**dataId, collections=collections)

    #select on star
    #src_cat = src_cat[(src_cat["extendedness"]==0) & (src_cat["parentSourceId"] == 0) ]
    #src_cat = src_cat[src_cat["extendedness"]==0 ]

    x1 = row_source['x_x']
    y1 = row_source['y_x']
    ra1= row_source['ra_x']
    dec1= row_source['dec_x']
    psfMag1  = row_source['psfMag_x']  
    psfSigma1 = row_source['psfSigma_x']
    psfMagDiffMmag1 = row_source['psfMagDiffMmag_x'] 
    apFlux_50_0_instFlux1 = row_source['apFlux_50_0_instFlux_x']
    apFlux_50_0_instFluxErr1 = row_source['apFlux_50_0_instFluxErr_x']


    x2 = row_source['x_y']
    y2 = row_source['y_y']
    ra2= row_source['ra_y']
    dec2= row_source['dec_y']
    psfMag2  = row_source['psfMag_y']  
    psfSigma2 = row_source['psfSigma_y']
    psfMagDiffMmag2 = row_source['psfMagDiffMmag_y'] 
    apFlux_50_0_instFlux2 = row_source['apFlux_50_0_instFlux_y']
    apFlux_50_0_instFluxErr2 = row_source['apFlux_50_0_instFluxErr_y']

    
    # searcch the neighbouring stars
    #separation = find_neighbourg(ra_target,dec_target,src_cat)
    #src_cat["sep"] = separation


    spherePointCenter = lsst.geom.SpherePoint(ra1*lsst.geom.degrees, dec1*lsst.geom.degrees)
    spt = lsst.geom.SpherePoint(ra2*lsst.geom.degrees, dec2*lsst.geom.degrees)
    ang = spherePointCenter.separation(spt)
    ang_arcsec = ang.asArcseconds()
    
   
    x0 = (x1+x2)/2.
    y0 = (y1+y2)/2.
    ra0 = (ra1+ra2)/2.
    dec0 = (dec1+dec2)/2.
    

    dx1 = x1-x0
    dy1 = y1-y0
    dx2 = x2-x0
    dy2 = y2-y0
    
    
    boxSize = int(max(np.abs(x1-x2),np.abs(y1-y2)))
    xmin= x0-boxSize
    xmax= x0+boxSize
    ymin= y0-boxSize
    ymax= y0+boxSize
        
    
    minBbox = geom.Point2I(int(x0) - boxSize ,int(y0) - boxSize)
    maxBbox = geom.Point2I(int(x0) + boxSize, int(y0) + boxSize)
    
    srcBbox = geom.Box2I(minBbox, maxBbox)


    row_source['x0'] = x0
    row_source['y0'] = y0
    row_source['xmin'] = xmin
    row_source['ymin'] = ymin
    row_source['xmax'] = xmax
    row_source['ymax'] = ymax
    row_source['ra0'] = ra0
    row_source['dec0'] = dec0

    row_source['x1'] = x1
    row_source['y1'] = y1
    row_source['dx1'] = dx1
    row_source['dy1'] = dy1
    row_source['ra1'] = ra1
    row_source['dec1'] = dec1

    row_source['x2'] = x2
    row_source['y2'] = y2
    row_source['dx2'] = dx2
    row_source['dy2'] = dy2
    row_source['ra2'] = ra2
    row_source['dec2'] = dec2
    
    
    # Make the cutout
    try:
        subimg = afwImage.ExposureF(calexp, srcBbox, afwImage.PARENT, True)
        #subimg2 = calexp.getImage().array[int(ySrc) - boxSize: int(ySrc) + boxSize+1, int(xSrc) - boxSize:int(xSrc) + boxSize+1 ]
        #subimgRgb = afwRgb.makeRGB(subimg.image.array, None, None, expMin, expMax - expMin, Q)
        #all_cutout.append(subimgRgb)
        cutout=subimg.image.array
        
    except Exception as inst:
        print(type(inst))    # the exception type
        print(inst.args)     # arguments stored in .args
        print(inst)     
        cutout = None

    df_pair = row_source.to_frame()
    return row_source,cutout



In [ ]:
def GetFluxes2StarsTwoBands(band1,band2,dfb1_b2,df_selectedvisits_b1,df_selectedvisits_b2):
    """
    Extract color difference from two bright object sources.
    band1 : first band tag 
    band2 : second band tag
    dfb1_b2 : pandas dataframe associating visit in band 1 to visit in band 2
    df_selectedvisits_b1: sources for object 1 and 2 in band 1
    df_selectedvisits_b2: sources for object 1 and 2 in band 2
    May plot some calexp cutout for checking
    """
    df_col_b1b2 = pd.DataFrame(columns = ["band1","exposure1","time1","psfMag11","psfMag12",
                                          "psfMagErr11","psfMagErr12",
                                          "apMag11","apMag12","apMagErr11","apMagErr12",
                                          "band2","exposure2","time2","psfMag21","psfMag22",
                                          "psfMagErr21","psfMagErr22",
                                          "apMag21","apMag22","apMagErr21","apMagErr22",
                                          "dt","airmassb1","airmassb2",
                                          "x11","y11","x12","y12","x21","y21","x22","y22"])
    entrycount = 0
    for idx,key in enumerate(dfb1_b2.index):
        print(idx,key)
        visit1 = key[1]
        visit2 = dfb1_b2.loc[key]["visit_" + band2]
        time1 = dfb1_b2.loc[key]["time"]
        time2 = dfb1_b2.loc[key]["time_" + band2]
        dt = dfb1_b2.loc[key]["dtsec_" + band2]
        print(f"======================= night :: {key[0]} ==> ({band1},{band2}) = ({visit1},{visit2}) =============================" )
        #row_target = df_selectedvisits_b1[df_selectedvisits_b1.visit == visit1].iloc[0]
        #print(row_target)
        # the selected sources retrieved from the sources associated to that calexp
        try:
            row_pair_b1,cutoutb1 = getcutoutandimage(visit1,df_selectedvisits_b1) 
            row_pair_b2,cutoutb2 = getcutoutandimage(visit2,df_selectedvisits_b2) 

            airmassb1 = row_pair_b1["airmass_x"] 
            airmassb2 = row_pair_b2["airmass_y"] 

            # first indec band, second index star
            # Star 1 in band 1
            psfMag11 =  row_pair_b1["psfMag_x"]
            psfMagErr11 =  row_pair_b1["psfMagErr_x"]
            apMag11 =  -2.5*np.log10(row_pair_b1["apFlux_35_0_instFlux_x"])
            apMagErr11 =  2.5/np.log(10)*(row_pair_b1["apFlux_35_0_instFluxErr_x"]/row_pair_b1["apFlux_35_0_instFlux_x"])
            x11 = row_pair_b1["x_x"]
            y11 = row_pair_b1["y_x"]
            
            # Star 2 in band 1
            psfMag12 =  row_pair_b1["psfMag_y"]
            psfMagErr12 =  row_pair_b1["psfMagErr_y"]
            apMag12 =  -2.5*np.log10(row_pair_b1["apFlux_35_0_instFlux_y"])
            apMagErr12 =  2.5/np.log(10)*(row_pair_b1["apFlux_35_0_instFluxErr_x"]/row_pair_b1["apFlux_35_0_instFlux_y"])
            x12 = row_pair_b1["x_y"]
            y12 = row_pair_b1["y_y"]
            
            # Star 1 in band 2
            psfMag21 =  row_pair_b2["psfMag_x"]
            psfMagErr21 =  row_pair_b2["psfMagErr_x"]
            apMag21 =  -2.5*np.log10(row_pair_b2["apFlux_35_0_instFlux_x"])
            apMagErr21 =  2.5/np.log(10)*(row_pair_b2["apFlux_35_0_instFluxErr_x"]/row_pair_b2["apFlux_35_0_instFlux_x"])
            x21 = row_pair_b2["x_x"]
            y21 = row_pair_b2["y_x"]
            
            # Star 2 in band 2
            psfMag22 =  row_pair_b2["psfMag_y"]
            psfMagErr22 =  row_pair_b2["psfMagErr_y"]
            apMag22 =  -2.5*np.log10(row_pair_b2["apFlux_35_0_instFlux_y"])
            apMagErr22 =  2.5/np.log(10)*(row_pair_b2["apFlux_35_0_instFluxErr_x"]/row_pair_b2["apFlux_35_0_instFlux_y"])
            x22 = row_pair_b2["x_y"]
            y22 = row_pair_b2["y_y"]


            
            df_col_b1b2.loc[entrycount] = [band1,visit1,time1,psfMag11,psfMag12,psfMagErr11,psfMagErr12,apMag11,apMag12,apMagErr11,apMagErr12,
                                           band2,visit2,time2,psfMag21,psfMag22,psfMagErr21,psfMagErr22,apMag21,apMag22,apMagErr21,apMagErr22,dt,
                                           airmassb1,airmassb2,x11,y11,x12,y12,x21,y21,x22,y22]
            entrycount+=1

        
        except Exception as inst:
            #print(type(inst))    # the exception type
            #print(inst.args)     # arguments stored in .args
            print(inst)
            continue
      

        if idx%10 == 0:
            if cutoutb1 is not None and cutoutb2 is not None:
                NROWS = 1
                NCOLS = 2
                fig, (ax1,ax2) = plt.subplots(ncols=NCOLS,nrows=NROWS,figsize=(6*NCOLS,6*NROWS))
                plotcutout(row_pair_b1,cutoutb1,ax=ax1)
                plotcutout(row_pair_b2,cutoutb2,ax=ax2)
                plt.show()

    # band 1 : mag_star1-mag_star2 
    df_col_b1b2["deltapsfmag_s12b1"] = (df_col_b1b2["psfMag11"] - df_col_b1b2["psfMag12"])
    df_col_b1b2["deltapsfmagErr_s12b1"] = np.sqrt(df_col_b1b2["psfMagErr11"]**2 + df_col_b1b2["psfMagErr12"]**2)

    df_col_b1b2["deltaapmag_s12b1"] = (df_col_b1b2["apMag11"] - df_col_b1b2["apMag12"])
    df_col_b1b2["deltaapmagErr_s12b1"] = np.sqrt(df_col_b1b2["apMagErr11"]**2 + df_col_b1b2["apMagErr12"]**2)
    
    # band 2 : mag_star1-mag_star2 
    df_col_b1b2["deltapsfmag_s12b2"] = (df_col_b1b2["psfMag21"] - df_col_b1b2["psfMag22"])
    df_col_b1b2["deltapsfmagErr_s12b2"] = np.sqrt(df_col_b1b2["psfMagErr21"]**2 + df_col_b1b2["psfMagErr22"]**2)
    df_col_b1b2["deltaapmag_s12b2"] = (df_col_b1b2["apMag21"] - df_col_b1b2["apMag22"])
    df_col_b1b2["deltaapmagErr_s12b2"] = np.sqrt(df_col_b1b2["apMagErr21"]**2 + df_col_b1b2["apMagErr22"]**2)

    
    # Star 1 : mag_band1-mag_band2 
    df_col_b1b2["psfcol12_s1"] = (df_col_b1b2["psfMag11"] - df_col_b1b2["psfMag21"])
    df_col_b1b2["psfcol12Err_s1"] = np.sqrt(df_col_b1b2["psfMagErr11"]**2 + df_col_b1b2["psfMagErr21"]**2)
    df_col_b1b2["apcol12_s1"] = (df_col_b1b2["apMag11"] - df_col_b1b2["apMag21"])
    df_col_b1b2["apcol12Err_s1"] = np.sqrt(df_col_b1b2["apMagErr11"]**2 + df_col_b1b2["apMagErr21"]**2)

    
    # Star 2 : mag_band1-mag_band2 
    df_col_b1b2["psfcol12_s2"] = (df_col_b1b2["psfMag12"] - df_col_b1b2["psfMag22"])
    df_col_b1b2["psfcol12Err_s2"] = np.sqrt(df_col_b1b2["psfMagErr12"]**2 + df_col_b1b2["psfMagErr22"]**2)
    df_col_b1b2["apcol12_s2"] = (df_col_b1b2["apMag12"] - df_col_b1b2["apMag22"])
    df_col_b1b2["apcol12Err_s2"] = np.sqrt(df_col_b1b2["apMagErr12"]**2 + df_col_b1b2["apMagErr22"]**2)

    
    # relative color star1 - star2
    df_col_b1b2["psfcol12_s12"] = np.abs(df_col_b1b2["psfcol12_s1"] - df_col_b1b2["psfcol12_s2"])
    df_col_b1b2["psfcol12Err_s12"] = np.sqrt(df_col_b1b2["psfcol12Err_s1"]**2 + df_col_b1b2["psfcol12Err_s2"]**2)
    df_col_b1b2["apcol12_s12"] = np.abs(df_col_b1b2["apcol12_s1"] - df_col_b1b2["apcol12_s2"])
    df_col_b1b2["apcol12Err_s12"] = np.sqrt(df_col_b1b2["apcol12Err_s1"]**2 + df_col_b1b2["apcol12Err_s2"]**2)

     # Maximum magnitude in band b1 and band b2
    df_col_b1b2["psfmagmax_s12b1"] = np.fmax(df_col_b1b2["psfMag11"],df_col_b1b2["psfMag12"])
    df_col_b1b2["psfmagmax_s12b2"] = np.fmax(df_col_b1b2["psfMag21"],df_col_b1b2["psfMag22"])

    df_col_b1b2["apmagmax_s12b1"] = np.fmax(df_col_b1b2["apMag11"],df_col_b1b2["apMag12"])
    df_col_b1b2["apmagmax_s12b2"] = np.fmax(df_col_b1b2["apMag21"],df_col_b1b2["apMag22"])
    
    
    return df_col_b1b2


In [ ]:
def GetFluxes2StarsTwoBandsNoCutout(band1,band2,dfb1_b2,df_selectedvisits_b1,df_selectedvisits_b2):
    """
    Extract color difference from two bright object sources.
    band1 : first band tag 
    band2 : second band tag
    dfb1_b2 : pandas dataframe associating visit in band 1 to visit in band 2
    df_selectedvisits_b1: sources for object 1 and 2 in band 1
    df_selectedvisits_b2: sources for object 1 and 2 in band 2
    NO NOT plot some calexp cutout contrary to above function
    """
    df_col_b1b2 = pd.DataFrame(columns = ["band1","exposure1","time1","psfMag11","psfMag12",
                                          "psfMagErr11","psfMagErr12",
                                          "apMag11","apMag12","apMagErr11","apMagErr12",
                                          "band2","exposure2","time2","psfMag21","psfMag22",
                                          "psfMagErr21","psfMagErr22",
                                          "apMag21","apMag22","apMagErr21","apMagErr22",
                                          "dt","airmassb1","airmassb2",
                                          "x11","y11","x12","y12","x21","y21","x22","y22"                                         
                                         ])
    
    entrycount = 0
    for idx,key in enumerate(dfb1_b2.index):
        print(idx,key)
        visit1 = key[1]
        visit2 = dfb1_b2.loc[key]["visit_" + band2]
        time1 = dfb1_b2.loc[key]["time"]
        time2 = dfb1_b2.loc[key]["time_" + band2]
        dt = dfb1_b2.loc[key]["dtsec_" + band2]
        print(f"======================= night :: {key[0]} ==> ({band1},{band2}) = ({visit1},{visit2}) =============================" )
        #row_target = df_selectedvisits_b1[df_selectedvisits_b1.visit == visit1].iloc[0]
        #print(row_target)
        # the selected sources retrieved from the sources associated to that calexp
        try:
            #row_pair_b1,cutoutb1 = getcutoutandimage(visit1,df_selectedvisits_b1) 
            #row_pair_b2,cutoutb2 = getcutoutandimage(visit2,df_selectedvisits_b2) 
            row_pair_b1  = df_selectedvisits_b1[df_selectedvisits_b1.visit == visit1].iloc[0]  
            row_pair_b2  = df_selectedvisits_b2[df_selectedvisits_b2.visit == visit2].iloc[0]  

            airmassb1 = row_pair_b1["airmass_x"] 
            airmassb2 = row_pair_b2["airmass_y"] 
            
            # first indec band, second index star
            # Star 1 in band 1
            psfMag11 =  row_pair_b1["psfMag_x"]
            psfMagErr11 =  row_pair_b1["psfMagErr_x"]    
            apMag11 =  -2.5*np.log10(row_pair_b1["apFlux_35_0_instFlux_x"])
            apMagErr11 =  2.5/np.log(10.)*(row_pair_b1["apFlux_35_0_instFluxErr_x"]/row_pair_b1["apFlux_35_0_instFlux_x"])
            x11 = row_pair_b1["x_x"]
            y11 = row_pair_b1["y_x"]
            
            # Star 2 in band 1
            psfMag12 =  row_pair_b1["psfMag_y"]
            psfMagErr12 =  row_pair_b1["psfMagErr_y"]
            apMag12 =  -2.5*np.log10(row_pair_b1["apFlux_35_0_instFlux_y"])
            apMagErr12 =  2.5/np.log(10.)*(row_pair_b1["apFlux_35_0_instFluxErr_x"]/row_pair_b1["apFlux_35_0_instFlux_y"])
            x12 = row_pair_b1["x_y"]
            y12 = row_pair_b1["y_y"]
            
            # Star 1 in band 2
            psfMag21 =  row_pair_b2["psfMag_x"]
            psfMagErr21 =  row_pair_b2["psfMagErr_x"]
            apMag21 =  -2.5*np.log10(row_pair_b2["apFlux_35_0_instFlux_x"])
            apMagErr21 =  2.5/np.log(10.)*(row_pair_b2["apFlux_35_0_instFluxErr_x"]/row_pair_b2["apFlux_35_0_instFlux_x"])
            x21 = row_pair_b2["x_x"]
            y21 = row_pair_b2["y_x"]

            
            # Star 2 in band 2
            psfMag22 =  row_pair_b2["psfMag_y"]
            psfMagErr22 =  row_pair_b2["psfMagErr_y"]
            apMag22 =  -2.5*np.log10(row_pair_b2["apFlux_35_0_instFlux_y"])
            apMagErr22 =  2.5/np.log(10.)*(row_pair_b2["apFlux_35_0_instFluxErr_x"]/row_pair_b2["apFlux_35_0_instFlux_y"])
            x22 = row_pair_b2["x_y"]
            y22 = row_pair_b2["y_y"]


            
            df_col_b1b2.loc[entrycount] = [band1,visit1,time1,psfMag11,psfMag12,psfMagErr11,psfMagErr12,apMag11,apMag12,apMagErr11,apMagErr12,
                                           band2,visit2,time2,psfMag21,psfMag22,psfMagErr21,psfMagErr22,apMag21,apMag22,apMagErr21,apMagErr22,dt,airmassb1,airmassb2,
                                           x11,y11,x12,y12,x21,y21,x22,y22]

            entrycount+=1
        except Exception as inst:
            #print(type(inst))    # the exception type
            #print(inst.args)     # arguments stored in .args
            print(inst)
            continue

    # band 1 : mag_star1-mag_star2 
    df_col_b1b2["deltapsfmag_s12b1"] = (df_col_b1b2["psfMag11"] - df_col_b1b2["psfMag12"])
    df_col_b1b2["deltapsfmagErr_s12b1"] = np.sqrt(df_col_b1b2["psfMagErr11"]**2 + df_col_b1b2["psfMagErr12"]**2)

    df_col_b1b2["deltaapmag_s12b1"] = (df_col_b1b2["apMag11"] - df_col_b1b2["apMag12"])
    df_col_b1b2["deltaapmagErr_s12b1"] = np.sqrt(df_col_b1b2["apMagErr11"]**2 + df_col_b1b2["apMagErr12"]**2)
    
    # band 2 : mag_star1-mag_star2 
    df_col_b1b2["deltapsfmag_s12b2"] = (df_col_b1b2["psfMag21"] - df_col_b1b2["psfMag22"])
    df_col_b1b2["deltapsfmagErr_s12b2"] = np.sqrt(df_col_b1b2["psfMagErr21"]**2 + df_col_b1b2["psfMagErr22"]**2)
    df_col_b1b2["deltaapmag_s12b2"] = (df_col_b1b2["apMag21"] - df_col_b1b2["apMag22"])
    df_col_b1b2["deltaapmagErr_s12b2"] = np.sqrt(df_col_b1b2["apMagErr21"]**2 + df_col_b1b2["apMagErr22"]**2)

    
    # Star 1 : mag_band1-mag_band2 
    df_col_b1b2["psfcol12_s1"] = (df_col_b1b2["psfMag11"] - df_col_b1b2["psfMag21"])
    df_col_b1b2["psfcol12Err_s1"] = np.sqrt(df_col_b1b2["psfMagErr11"]**2 + df_col_b1b2["psfMagErr21"]**2)
    df_col_b1b2["apcol12_s1"] = (df_col_b1b2["apMag11"] - df_col_b1b2["apMag21"])
    df_col_b1b2["apcol12Err_s1"] = np.sqrt(df_col_b1b2["apMagErr11"]**2 + df_col_b1b2["apMagErr21"]**2)

    
    # Star 2 : mag_band1-mag_band2 
    df_col_b1b2["psfcol12_s2"] = (df_col_b1b2["psfMag12"] - df_col_b1b2["psfMag22"])
    df_col_b1b2["psfcol12Err_s2"] = np.sqrt(df_col_b1b2["psfMagErr12"]**2 + df_col_b1b2["psfMagErr22"]**2)
    df_col_b1b2["apcol12_s2"] = (df_col_b1b2["apMag12"] - df_col_b1b2["apMag22"])
    df_col_b1b2["apcol12Err_s2"] = np.sqrt(df_col_b1b2["apMagErr12"]**2 + df_col_b1b2["apMagErr22"]**2)

    
    # absolute relative color star1 - star2
    df_col_b1b2["psfcol12_s12"] = np.abs(df_col_b1b2["psfcol12_s1"] - df_col_b1b2["psfcol12_s2"])
    df_col_b1b2["psfcol12Err_s12"] = np.sqrt(df_col_b1b2["psfcol12Err_s1"]**2 + df_col_b1b2["psfcol12Err_s2"]**2)
    df_col_b1b2["apcol12_s12"] = np.abs(df_col_b1b2["apcol12_s1"] - df_col_b1b2["apcol12_s2"])
    df_col_b1b2["apcol12Err_s12"] = np.sqrt(df_col_b1b2["apcol12Err_s1"]**2 + df_col_b1b2["apcol12Err_s2"]**2)

    # Maximum magnitude in band b1 and band b2
    df_col_b1b2["psfmagmax_s12b1"] = np.fmax(df_col_b1b2["psfMag11"],df_col_b1b2["psfMag12"])
    df_col_b1b2["psfmagmax_s12b2"] = np.fmax(df_col_b1b2["psfMag21"],df_col_b1b2["psfMag22"])

    df_col_b1b2["apmagmax_s12b1"] = np.fmax(df_col_b1b2["apMag11"],df_col_b1b2["apMag12"])
    df_col_b1b2["apmagmax_s12b2"] = np.fmax(df_col_b1b2["apMag21"],df_col_b1b2["apMag22"])
    


    return df_col_b1b2

In [ ]:
def closesttimespectrophotom(df_photpair,df_spec):
    """
    Search the closest spectroscopic time for a photometric light curve

    - calculate with assign
    df_spec[["Time"]].assign( ddt = lambda row : np.abs((row["Time"].loc[0].replace(tzinfo=None) - df_photpair["time1"].loc[1]).total_seconds()))
    - calculate with apply two quantities : dt_abs, dt_rel    
    df_spec[["dt_abs","td_rel"]]=df_spec[["Time"]].apply( lambda row : [ np.abs((row["Time"].replace(tzinfo=None) - df_photpair["time1"].loc[1]).total_seconds()),(row["Time"].replace(tzinfo=None) - df_photpair["time1"].loc[1]).total_seconds()],axis=1,result_type='expand')
    """
    d1 =df_photpair.copy(deep=True)
    d2 =df_spec.copy(deep=True)
    all_samples = []
    df = pd.DataFrame(index=d1.index,columns=["time1","time2","deltat"])
    
    # loop on light curve datapoints times
    all_spectro_associated = []
    for key in d1.index:
        # take the time of the first source 
        # remember dt must be small
        tphotometric = df_photpair["time1"].loc[key]
        df_spec_copy = df_spec.copy(deep=True)
        # compute relative and absolute time difference of spectro wrt photmetry
        df_spec_copy[["dt_abs_specphot","td_rel_specphot"]]=df_spec_copy[["Time"]].apply( lambda row : [ np.abs((row["Time"].replace(tzinfo=None) - tphotometric).total_seconds()),(row["Time"].replace(tzinfo=None) - tphotometric ).total_seconds()],axis=1,result_type='expand')
        # find the minimum on absolute time difference between spectro and photometry
        df_spec_sel = df_spec_copy[ df_spec_copy["dt_abs_specphot"] == df_spec_copy["dt_abs_specphot"].min()]
        all_spectro_associated.append(df_spec_sel)

    # merge all datapoint on single dataframe
    df_spec_sel_concat = pd.concat(all_spectro_associated)
    df_spec_sel_concat.reset_index(level=0, inplace=True)

    # set nan if more than 12*3600 seconds outside
    
    return df_spec_sel_concat


## Selected visits

### Spectroscopy

In [ ]:
if version_results == "":
    atmfilename = "data/spectro/auxtel_atmosphere_202301_v3.1.0_doSensorFlat_rebin2_testWithMaskedEdges_newBoundaries_newPolysRescaled_newFitBounds_adjustA1_lockedOrder2_removeThroughputTails_2.npy"
elif version_results == "v2":
    atmfilename = "data/spectro/auxtel_atmosphere_202301_v3.1.0_doSensorFlat_rebin2_lockedOrder2_FixA1_FixA2_FitAngstrom_FixA1_FixA2_FitAngstrom_WithGaia_freePressure_newThroughput6_BG40Scaled1.09_PeekFinder.npy"

In [ ]:
specdata = np.load(atmfilename,allow_pickle=True)

In [ ]:
df_spec = pd.DataFrame(specdata)
df_spec

In [ ]:
df_spec["Time"] = pd.to_datetime(df_spec["DATE-OBS"])
#df_spec["PWV [mm]_x"]

In [ ]:
from matplotlib.dates import DateFormatter
date_form = DateFormatter("%y-%m-%dT%H:%M")
fig,axs = plt.subplots(2,1,figsize=(14,8))
ax1,ax2  = axs
df_spec.plot(x="Time",y="PWV [mm]_x",ax=ax1,marker='+',c="r",lw=0.0,ms=10,grid=True,label="PWV_x")
ax1.set_ylabel("PWV [mm]_x")
df_spec.plot(x="Time",y="PWV [mm]_y",ax=ax2,marker='+',c="b",lw=0.0,ms=10,grid=True,label="PWV_y")
ax1.set_xlabel("time")
ax1.xaxis.set_major_formatter(date_form)
ax2.set_xlabel("time")
ax2.set_ylabel("PWV [mm]_y")
ax2.xaxis.set_major_formatter(date_form)
ax1.set_title("Precipibable water vapor meas by holo vs time")
plt.tight_layout()

## Photometry

#### read list of all possible pairs

In [ ]:
df_allpair = pd.read_csv("all_pairs.csv",index_col=0)
df_allpair.head()

In [ ]:
Nallpairs = len(df_allpair)

In [ ]:
fig,axs = plt.subplots(1,1,figsize=(6,4))
ax1 = axs
df_allpair["sep"].hist(bins=50,range=(0.,750),ax=ax1,histtype="step",color="b",lw=2);
image_size = 0.1*4000.
ax1.axvline(image_size,color="k")
ax1.set_xlabel("$\\theta$ (arcsec)")
ax1.set_title(f"Bright sources angular correlation in tract {tract}")
textstr = f"Npairs = {Nallpairs}"
ax1.text(0.65, 0.95, textstr, transform=ax1.transAxes, fontsize=14,verticalalignment='top', bbox=props)


### Select the pairs

In [ ]:
#sep_max = 100.
df_pair = df_allpair[df_allpair["sep"]<sep_max]
Npairs = len(df_pair)

In [ ]:
fig,axs = plt.subplots(1,1,figsize=(8,6))
ax1 = axs
df_allpair["sep"].hist(bins=50,range=(0.,750),ax=ax1,histtype="step",ls="-.",color="b",lw=2);
df_pair["sep"].hist(bins=50,range=(0.,750),ax=ax1,histtype="step",color="b",lw=2);
image_size = 0.1*4000.
ax1.axvline(image_size,color="k")
ax1.set_xlabel("$\\theta$ (arcsec)")
ax1.set_title(f"Bright sources angular correlation in tract {tract}")

textstr1 = f"N all pairs      = {Nallpairs}"
textstr2 = f"N pairs selected = {Npairs}"
textstr = "\n".join((textstr1,textstr2))
ax1.text(0.6, 0.95, textstr, transform=ax1.transAxes, fontsize=14,verticalalignment='top', bbox=props)

## Read pairs

In [ ]:
#path_input = f"data_coloredlightcurvesZ_Y_thmax_{sep_max:.0f}"
if version_results == "":
    path_input = f"data_coloredlightcurvesZ_Y_thmax_{sep_max:.0f}_tmax_{sep_time:.0f}"
    
elif version_results == "v2":
    path_input = f"data_coloredlightcurvesZ_Y_thmax_{sep_max:.0f}_tmax_{sep_time:.0f}_{version_results}"
    
if not os.path.exists(path_input):
    print(f"Missing directory {path_input}  ==> Must run notebook LoopTwinSourcesMultiColorLightCurveToFitsfromPairs-Auxtel-ColorZY.ipynb")
    assert False

### Read summary

- Note summary has been produced by **LookTwinSourcesMultiColorLightCurveToFitsfromPairs-Auxtel-ColorZY.ipynb**

In [ ]:
#df_summary = pd.read_csv(os.path.join(path_input,f"photompairsummaryZ_Y_thmax_{sep_max:.0f}.csv"),index_col=0)

if version_results == "":
    df_summary = pd.read_csv(os.path.join(path_input,f"photompairsummaryZ_Y_thmax_{sep_max:.0f}_tmax_{sep_time:.0f}.csv"),index_col=0)
elif version_results == "v2":
    df_summary = pd.read_csv(os.path.join(path_input,f"photompairsummaryZ_Y_thmax_{sep_max:.0f}_tmax_{sep_time:.0f}_{version_results}.csv"),index_col=0)
    

In [ ]:
df_summary

## Read all colors_diff

In [ ]:
NPairs = int(df_summary["idx_pair"].max())

In [ ]:
all_colors_diff = []
for idx in range(NPairs):
    num_str = str(idx).zfill(4)
    input_filename = f"color_lightcurveZ_Y_{num_str}_thmax_{sep_max:.0f}.csv"
    if version_results == "":
        input_filename = f"color_lightcurveZ_Y_{num_str}_thmax_{sep_max:.0f}_tmax_{sep_time:.0f}.csv"
    elif version_results == "v2":
        input_filename = f"color_lightcurveZ_Y_{num_str}_thmax_{sep_max:.0f}_tmax_{sep_time:.0f}_{version_results}.csv"
        
    input_fullfilename = os.path.join(path_input,input_filename)
    df_col = pd.read_csv(input_fullfilename,index_col=0)
    if len(df_col) > 0:
        # convert date string in datetime format for plotting date
        df_col["time1"] = df_col.apply( lambda row : pd.to_datetime(row["time1"]),axis=1,result_type="expand")
        df_col["time2"] = df_col.apply( lambda row : pd.to_datetime(row["time2"]),axis=1,result_type="expand")
        df_col["Time"] = df_col.apply( lambda row : pd.to_datetime(row["Time"]),axis=1,result_type="expand")
        df_col["DATE-OBS"] = df_col.apply( lambda row : pd.to_datetime(row["DATE-OBS"]),axis=1,result_type="expand")
       
    all_colors_diff.append(df_col)

In [ ]:
Npairs = len(all_colors_diff)

In [ ]:
textstr = '\n'.join((
    r'$N_{pairs}=%.0f$' % (Npairs, ),
    r'$\theta_{sep}<%.0f$ arcsec' % (sep_max, )))

In [ ]:
all_colors_diff[0][["time1","time2","dt","DATE-OBS","Time","dt_abs_specphot","td_rel_specphot"]] 

### Check good/bad light curves

In [ ]:
pd.options.display.max_rows = 300
df_summary["drefcol_psf_ap"] = df_summary["refcol_psf_md"] - df_summary["refcol_ap_md"]
df_summary

In [ ]:
df_summary[df_summary.refcol_ap_md < 0.02]

In [ ]:
df_summary[df_summary.refcol_ap_md > 0.2]

### Plot all relative colors histograms

In [ ]:
cmap = mpl.colormaps['jet']
colors_forplots = cmap(np.linspace(0, 1, Npairs))

In [ ]:
fig,(ax1,ax2) = plt.subplots(2,1,figsize=(12,6))
all_objectplotcolor = [] # compute median color per object
all_Ntimepoints = []
for idx,df in enumerate(all_colors_diff):
    df["psfcol12_s12"].hist(bins=100,range=(0,0.5),ax=ax1,histtype="step",color=colors_forplots[idx],lw=3)
ax1.set_title("relative color diff with psf flux")
ax1.text(0.8, 0.9, textstr, transform=ax1.transAxes, fontsize=14,verticalalignment='top', bbox=props)

for idx,df in enumerate(all_colors_diff):
    df["apcol12_s12"].hist(bins=100,range=(0,0.5),ax=ax2,histtype="step",color=colors_forplots[idx],lw=3)
    data =  df["apcol12_s12"].values
    if len(data)>=1:
        data= data[np.isfinite(data)]
        all_objectplotcolor.append(np.median(data))
        all_Ntimepoints.append(len(data))
    else:
        all_objectplotcolor.append(0.)
        
ax1.set_xlabel("PSF : $|(Z-Y)_1 - (Z-Y)_2|$")
ax2.set_xlabel("AP35 : $|(Z-Y)_1 - (Z-Y)_2|$")

ax2.set_title("relative color diff with Ap35 flux")
ax2.text(0.8, 0.9, textstr, transform=ax2.transAxes, fontsize=14,verticalalignment='top', bbox=props)
plt.tight_layout()

## Plots

In [ ]:
# wavelength bin colors
jet = plt.get_cmap('jet')
cNorm = mpl.colors.Normalize(vmin=0, vmax=0.35)
scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)
all_colors = scalarMap.to_rgba(all_objectplotcolor, alpha=1)


https://matplotlib.org/stable/users/explain/colors/colorbar_only.html

In [ ]:
fig, ax = plt.subplots(figsize=(6, 1), layout='constrained')
cmap = mpl.cm.jet
norm = mpl.colors.Normalize(vmin=0, vmax=0.4)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),cax=ax, orientation='horizontal', label='$|(Z_1-Y_1)-(Z_2-Y_2)|$')


In [ ]:
from matplotlib.dates import DateFormatter
#date_form = DateFormatter("%y-%m-%dT%H:%M")
date_form = DateFormatter("%y-%m-%d")
#fig,axs = plt.subplots(2,1,figsize=(14,8),sharex=True)

fig = plt.figure(figsize=(14,10),constrained_layout=True)
gs = GridSpec(2, 1, wspace=0, hspace=0.1, height_ratios=[1,2],figure=fig)

ax1 = fig.add_subplot(gs[0])
ax2 = fig.add_subplot(gs[1],sharex=ax1)


df_spec.plot(x="Time",y="PWV [mm]_x",ax=ax1,marker='+',c="r",lw=0.0,grid=True,label="spectro PWV_x")

all_tmin = []
all_tmax = []

for idx,df_col in enumerate(all_colors_diff):
    the_color = all_colors[idx]

    if(len(df_col)>5):

        #print(idx,df_col)
        df_col.plot(x="time1",y="psfcol12_s12",ax=ax2,marker="o",c=the_color,lw=0.5,grid=True,legend=False)
        #ax2.errorbar(df_col.time1,df_col.psfcol12_s12,yerr=df_col.psfcol12Err_s12,color=the_color,ecolor="k",fmt="o")

        tmin = df_col["time1"].min()
        tmax = df_col["time1"].max()
        
        dt = tmax-tmin
        tmin = tmin-dt/10
        tmax = tmax+dt/10
        all_tmin.append(tmin)
        all_tmax.append(tmax)

ax2.set_xlabel("time")
ax2.xaxis.set_major_formatter(date_form)

the_tmin=np.min(all_tmin)
the_tmax=np.max(all_tmax)

ax2.set_xlim(the_tmin,the_tmax)
ax2.set_title("Auxtel PSF Photometry: $|(Z-Y)_1-(Z-Y)_2|$")
ax1.legend(loc="upper left")
ax1.set_ylabel("PWV (mm)")
ax1.set_title("Auxtel Spectroscopy : precipitable water vapor")
#ax2.legend(loc="upper left")
ax2.set_ylabel("PSF: $|(Z-Y)_1-(Z-Y)_2|$")
ax2.set_ylim(0.,0.5)
ax2.text(0.01, 0.95, textstr, transform=ax2.transAxes, fontsize=14,verticalalignment='top', bbox=props)

title = suptitle
plt.suptitle(title,y=1)
plt.tight_layout()

In [ ]:
from matplotlib.dates import DateFormatter
#date_form = DateFormatter("%y-%m-%dT%H:%M")
date_form = DateFormatter("%y-%m-%d")
#fig,axs = plt.subplots(2,1,figsize=(14,8),sharex=True)

fig = plt.figure(figsize=(14,10),constrained_layout=True)
gs = GridSpec(4, 1, wspace=0, hspace=0.1, height_ratios=[1,1,1,1],figure=fig)

ax1 = fig.add_subplot(gs[0])
ax2 = fig.add_subplot(gs[1],sharex=ax1)
ax3 = fig.add_subplot(gs[2],sharex=ax1)
ax4 = fig.add_subplot(gs[3],sharex=ax1)

## ax1
df_spec.plot(x="Time",y="PWV [mm]_x",ax=ax1,marker='+',c="r",lw=0.0,grid=True,label="spectro PWV_x")

all_tmin = []
all_tmax = []

## ax2 ##

#Colormap
DCOLMIN = 0.2
DCOLMAX = 0.3
jet = plt.get_cmap('jet')
cNorm = mpl.colors.Normalize(vmin=DCOLMIN, vmax=DCOLMAX)
scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)

for idx,df_col in enumerate(all_colors_diff):
    if(len(df_col)>5):
        the_relativecolor = all_objectplotcolor[idx] 
        if the_relativecolor> DCOLMIN :
            the_color = scalarMap.to_rgba(the_relativecolor, alpha=1)
            df_col.plot(x="time1",y="psfcol12_s12",ax=ax2,marker="o",c=the_color,lw=0.5,grid=True,legend=False)
            ax2.errorbar(df_col.time1,df_col.psfcol12_s12,yerr=df_col.psfcol12Err_s12,color=the_color,ecolor="k",fmt="o")

            tmin = df_col["time1"].min()
            tmax = df_col["time1"].max()
            dt = tmax-tmin
            tmin = tmin-dt/10
            tmax = tmax+dt/10
            all_tmin.append(tmin)
            all_tmax.append(tmax)

ax2.set_ylim(DCOLMIN-0.1,DCOLMAX+0.15)

## ax3

#Colormap
DCOLMIN = 0.1
DCOLMAX = 0.2
jet = plt.get_cmap('jet')
cNorm = mpl.colors.Normalize(vmin=DCOLMIN, vmax=DCOLMAX)
scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)

for idx,df_col in enumerate(all_colors_diff):
    if(len(df_col)>5):
        the_relativecolor = all_objectplotcolor[idx] 
        if the_relativecolor> DCOLMIN  and  the_relativecolor<= DCOLMAX:
            the_color = scalarMap.to_rgba(the_relativecolor, alpha=1)
            df_col.plot(x="time1",y="psfcol12_s12",ax=ax3,marker="o",c=the_color,lw=0.5,grid=True,legend=False)
            ax3.errorbar(df_col.time1,df_col.psfcol12_s12,yerr=df_col.psfcol12Err_s12,color=the_color,ecolor="k",fmt="o")

            tmin = df_col["time1"].min()
            tmax = df_col["time1"].max()
            dt = tmax-tmin
            tmin = tmin-dt/10
            tmax = tmax+dt/10
            all_tmin.append(tmin)
            all_tmax.append(tmax)

ax3.set_ylim(DCOLMIN-0.1,DCOLMAX+0.15)
## ax4

#Colormap
DCOLMIN = 0.
DCOLMAX = 0.1
jet = plt.get_cmap('jet')
cNorm = mpl.colors.Normalize(vmin=DCOLMIN, vmax=DCOLMAX)
scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)

for idx,df_col in enumerate(all_colors_diff):
    if(len(df_col)>5):
        the_relativecolor = all_objectplotcolor[idx] 
        if the_relativecolor> DCOLMIN  and  the_relativecolor<= DCOLMAX:
            the_color = scalarMap.to_rgba(the_relativecolor, alpha=1)
            df_col.plot(x="time1",y="psfcol12_s12",ax=ax4,marker="o",c=the_color,lw=0.5,grid=True,legend=False)
            ax4.errorbar(df_col.time1,df_col.psfcol12_s12,yerr=df_col.psfcol12Err_s12,color=the_color,ecolor="k",fmt="o")

            tmin = df_col["time1"].min()
            tmax = df_col["time1"].max()
            dt = tmax-tmin
            tmin = tmin-dt/10
            tmax = tmax+dt/10
            all_tmin.append(tmin)
            all_tmax.append(tmax)

ax4.set_ylim(DCOLMIN-0.1,DCOLMAX+0.15)

ax4.set_xlabel("time")
ax4.xaxis.set_major_formatter(date_form)

the_tmin=np.min(all_tmin)
the_tmax=np.max(all_tmax)

ax4.set_xlim(the_tmin,the_tmax)
ax2.set_title("Auxtel PSF Photometry: $|(Z-Y)_1-(Z-Y)_2|$")
ax1.legend(loc="upper left")
ax1.set_ylabel("PWV (mm)")
ax1.set_title("Auxtel Spectroscopy : precipitable water vapor")
#ax2.legend(loc="upper left")
ax2.set_ylabel("PSF: $|(Z-Y)_1-(Z-Y)_2|$")
ax4.set_ylabel("PSF: $|(Z-Y)_1-(Z-Y)_2|$")
ax2.text(0.01, 0.95, textstr, transform=ax2.transAxes, fontsize=14,verticalalignment='top', bbox=props)


title = suptitle
plt.suptitle(title,y=1.)
plt.tight_layout()

In [ ]:
from matplotlib.dates import DateFormatter
#date_form = DateFormatter("%y-%m-%dT%H:%M")
date_form = DateFormatter("%y-%m-%d")
#fig,axs = plt.subplots(2,1,figsize=(14,8),sharex=True)

fig = plt.figure(figsize=(14,10),constrained_layout=True)
gs = GridSpec(2, 1, wspace=0, hspace=0.1, height_ratios=[1,2],figure=fig)

ax1 = fig.add_subplot(gs[0])
ax2 = fig.add_subplot(gs[1],sharex=ax1)


df_spec.plot(x="Time",y="PWV [mm]_x",ax=ax1,marker='+',c="r",lw=0.0,grid=True,label="spectro PWV_x")

all_tmin = []
all_tmax = []

for idx,df_col in enumerate(all_colors_diff):
    the_color = all_colors[idx]

    if(len(df_col)>5):
        df_col.plot(x="time1",y="apcol12_s12",ax=ax2,marker="o",c=the_color,lw=0.5,grid=True,legend=False)
        ax2.errorbar(df_col.time1,df_col.apcol12_s12,yerr=df_col.apcol12Err_s12,color=the_color,ecolor="k",fmt="o")

        tmin = df_col["time1"].min()
        tmax = df_col["time1"].max()
        dt = tmax-tmin
        tmin = tmin-dt/10
        tmax = tmax+dt/10
        all_tmin.append(tmin)
        all_tmax.append(tmax)

ax2.set_xlabel("time")
ax2.xaxis.set_major_formatter(date_form)

the_tmin=np.min(all_tmin)
the_tmax=np.max(all_tmax)

ax2.set_xlim(the_tmin,the_tmax)
ax2.set_title("Auxtel Aperture Photometry Ap35: $|(Z-Y)_1-(Z-Y)_2|$")
ax1.legend(loc="upper left")
ax1.set_ylabel("PWV (mm)")
ax1.set_title("Auxtel Spectroscopy : precipitable water vapor")
#ax2.legend(loc="upper left")
ax2.set_ylabel("Ap35: $|(Z-Y)_1-(Z-Y)_2|$")
ax2.set_ylim(0.,0.5)
ax2.text(0.01, 0.95, textstr, transform=ax2.transAxes, fontsize=14,verticalalignment='top', bbox=props)

title = suptitle
plt.suptitle(title,y=1.)
plt.tight_layout()

In [ ]:
from matplotlib.dates import DateFormatter
#date_form = DateFormatter("%y-%m-%dT%H:%M")
date_form = DateFormatter("%y-%m-%d")
#fig,axs = plt.subplots(2,1,figsize=(14,8),sharex=True)

fig = plt.figure(figsize=(14,10),constrained_layout=True)
gs = GridSpec(4, 1, wspace=0, hspace=0.1, height_ratios=[1,1,1,1],figure=fig)

ax1 = fig.add_subplot(gs[0])
ax2 = fig.add_subplot(gs[1],sharex=ax1)
ax3 = fig.add_subplot(gs[2],sharex=ax1)
ax4 = fig.add_subplot(gs[3],sharex=ax1)

## ax1
df_spec.plot(x="Time",y="PWV [mm]_x",ax=ax1,marker='+',c="r",lw=0.0,grid=True,label="spectro PWV_x")

all_tmin = []
all_tmax = []

## ax2 ##

#Colormap
DCOLMIN = 0.2
DCOLMAX = 0.3
jet = plt.get_cmap('jet')
cNorm = mpl.colors.Normalize(vmin=DCOLMIN, vmax=DCOLMAX)
scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)

for idx,df_col in enumerate(all_colors_diff):
    if(len(df_col)>5):
        the_relativecolor = all_objectplotcolor[idx] 
        if the_relativecolor> DCOLMIN :
            the_color = scalarMap.to_rgba(the_relativecolor, alpha=1)
            df_col.plot(x="time1",y="apcol12_s12",ax=ax2,marker="o",c=the_color,lw=0.5,grid=True,legend=False)
            ax2.errorbar(df_col.time1,df_col.apcol12_s12,yerr=df_col.apcol12Err_s12,color=the_color,ecolor="k",fmt="o")

            tmin = df_col["time1"].min()
            tmax = df_col["time1"].max()
            dt = tmax-tmin
            tmin = tmin-dt/10
            tmax = tmax+dt/10
            all_tmin.append(tmin)
            all_tmax.append(tmax)

ax2.set_ylim(DCOLMIN-0.1,DCOLMAX+0.15)

## ax3

#Colormap
DCOLMIN = 0.1
DCOLMAX = 0.2
jet = plt.get_cmap('jet')
cNorm = mpl.colors.Normalize(vmin=DCOLMIN, vmax=DCOLMAX)
scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)

for idx,df_col in enumerate(all_colors_diff):
    if(len(df_col)>5):
        the_relativecolor = all_objectplotcolor[idx] 
        if the_relativecolor> DCOLMIN  and  the_relativecolor<= DCOLMAX:
            the_color = scalarMap.to_rgba(the_relativecolor, alpha=1)
            df_col.plot(x="time1",y="apcol12_s12",ax=ax3,marker="o",c=the_color,lw=0.5,grid=True,legend=False)
            ax3.errorbar(df_col.time1,df_col.apcol12_s12,yerr=df_col.apcol12Err_s12,color=the_color,ecolor="k",fmt="o")

            tmin = df_col["time1"].min()
            tmax = df_col["time1"].max()
            dt = tmax-tmin
            tmin = tmin-dt/10
            tmax = tmax+dt/10
            all_tmin.append(tmin)
            all_tmax.append(tmax)

ax3.set_ylim(DCOLMIN-0.1,DCOLMAX+0.15)
## ax4

#Colormap
DCOLMIN = 0.
DCOLMAX = 0.1
jet = plt.get_cmap('jet')
cNorm = mpl.colors.Normalize(vmin=DCOLMIN, vmax=DCOLMAX)
scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)

for idx,df_col in enumerate(all_colors_diff):
    if(len(df_col)>5):
        the_relativecolor = all_objectplotcolor[idx] 
        if the_relativecolor> DCOLMIN  and  the_relativecolor<= DCOLMAX:
            the_color = scalarMap.to_rgba(the_relativecolor, alpha=1)
            df_col.plot(x="time1",y="apcol12_s12",ax=ax4,marker="o",c=the_color,lw=0.5,grid=True,legend=False)
            ax4.errorbar(df_col.time1,df_col.apcol12_s12,yerr=df_col.apcol12Err_s12,color=the_color,ecolor="k",fmt="o")

            tmin = df_col["time1"].min()
            tmax = df_col["time1"].max()
            dt = tmax-tmin
            tmin = tmin-dt/10
            tmax = tmax+dt/10
            all_tmin.append(tmin)
            all_tmax.append(tmax)

ax4.set_ylim(DCOLMIN-0.1,DCOLMAX+0.15)

ax4.set_xlabel("time")
ax4.xaxis.set_major_formatter(date_form)

the_tmin=np.min(all_tmin)
the_tmax=np.max(all_tmax)

ax4.set_xlim(the_tmin,the_tmax)
ax2.set_title("Auxtel Aperture Photometry: $|(Z-Y)_1-(Z-Y)_2|$")
ax1.legend(loc="upper left")
ax1.set_ylabel("PWV (mm)")
ax1.set_title("Auxtel Spectroscopy : precipitable water vapor")
#ax2.legend(loc="upper left")
ax2.set_ylabel("Ap35: $|(Z-Y)_1-(Z-Y)_2|$")
ax4.set_ylabel("Ap35: $|(Z-Y)_1-(Z-Y)_2|$")
ax2.text(0.01, 0.95, textstr, transform=ax2.transAxes, fontsize=14,verticalalignment='top', bbox=props)


title = suptitle
plt.suptitle(title,y=1.)
plt.tight_layout()

In [ ]:
from matplotlib.dates import DateFormatter
#date_form = DateFormatter("%y-%m-%dT%H:%M")
date_form = DateFormatter("%y-%m-%d")
#fig,axs = plt.subplots(2,1,figsize=(14,8),sharex=True)

fig,ax1 = plt.subplots(1,1,figsize=(14,5),constrained_layout=True)


df_spec.plot(x="Time",y="PWV [mm]_x",ax=ax1,marker='+',c="r",lw=0.0,ms=8,grid=True,label="Spectro measurement")

all_tmin = []
all_tmax = []

for idx,df_col in enumerate(all_colors_diff):
    the_color = all_colors[idx]

    
    
    try:
        if idx == 0:
            df_col.plot(x="Time",y="PWV [mm]_x",ax=ax1,marker="o",c="b",lw=0.,ms=8,grid=True,label="Photometric observations")
        else:
            df_col.plot(x="Time",y="PWV [mm]_x",ax=ax1,marker="o",c="b",lw=0.,ms=8,grid=True,legend=False)

        if len(df_col)>5:
            tmin = df_col["time1"].min()
            tmax = df_col["time1"].max()
           
            dt = tmax-tmin
            tmin = tmin-dt/10
            tmax = tmax+dt/10
            all_tmin.append(tmin)
            all_tmax.append(tmax)
            #print(idx,tmin,tmax,dt)
    except:
        print(f"Unexpected error at idx ={idx} :", sys.exc_info()[0])


ax1.set_xlabel("time",fontsize=13)
ax1.xaxis.set_major_formatter(date_form)
#ax1.legend(False)

the_tmin=np.min(all_tmin)
the_tmax=np.max(all_tmax)

ax1.set_xlim(the_tmin,the_tmax)
ax1.set_ylabel("PWV (mm)",fontsize=14)
ax1.set_title("Auxtel Coincidence between spectro-photometry observations",fontsize=20)
title = suptitle
plt.suptitle(title,y=1)
plt.tight_layout()

## Check if spread varies with (Z-Y) relative difference

In [ ]:
df_summary

In [ ]:
for index in range(len(df_summary["idx_pair"])):
    #print(df_summary.iloc[index][["idx_pair","idx_obj1","idx_obj2"]])
    text = "{:}){:}({:},{:})".format(index, int(df_summary.iloc[index]["idx_pair"]), int(df_summary.iloc[index]["idx_obj1"]),int(df_summary.iloc[index]["idx_obj2"]))
    #print(text)

In [ ]:
fig, (ax1,ax2,ax3) = plt.subplots(3,1,figsize=(18,30))
title = f"AuxtelLightCurves"


sc1=df_summary.plot.scatter(x= "muclip",y="sigmaclip",ax=ax1,s=80, c= "sep",cmap="jet", label="$|(Z-Y)_1 - (Y-Z)_2|$", grid=True,colorbar=True)
sc2=df_summary.plot.scatter(x= "muclip",y="sigmaclip",ax=ax2,s=80, c= "nclip",cmap="jet", label="$|(Z-Y)_1 - (Y-Z)_2|$", grid=True,colorbar=True)
sc3=df_summary.plot.scatter(x= "muclip",y="sigmaclip",ax=ax3,s=80, c= "magmax",cmap="jet", label="$|(Z-Y)_1 - (Y-Z)_2|$", grid=True,colorbar=True)

ax1.set_title(title)
ax1.set_title("$\\sigma (|(Z-Y)_1 - (Y-Z)_2|) vs \overline{(|(Z-Y)_1 - (Y-Z)_2|}$")
ax1.set_xlabel("$\overline{(|(Z-Y)_1 - (Y-Z)_2|}$ (mag)")
ax1.set_ylabel("$\\sigma (|(Z-Y)_1 - (Y-Z)_2|) $ (mag)")



ax2.set_title("$\\sigma (|(Z-Y)_1 - (Y-Z)_2|) vs \overline{(|(Z-Y)_1 - (Y-Z)_2|}$")
ax2.set_xlabel("$\overline{(|(Z-Y)_1 - (Y-Z)_2|}$ (mag)")
ax2.set_ylabel("$\\sigma (|(Z-Y)_1 - (Y-Z)_2|) $ (mag)")


ax3.set_title("$\\sigma (|(Z-Y)_1 - (Y-Z)_2|) vs \overline{(|(Z-Y)_1 - (Y-Z)_2|}$")
ax3.set_xlabel("$\overline{(|(Z-Y)_1 - (Y-Z)_2|}$ (mag)")
ax3.set_ylabel("$\\sigma (|(Z-Y)_1 - (Y-Z)_2|) $ (mag)")

#cbar1 = plt.colorbar(ax1.collections[0],ax=ax1,orientation = 'horizontal',shrink=0.6)
#cbar1 = plt.colorbar(ax1.collections[0],ax=ax1,orientation = 'horizontal',shrink=0.6)
#cbar1.ax.set_xlabel('angular separation (arcsec)',rotation=0)

#cbar2 = plt.colorbar(ax2.collections[0],ax=ax2,orientation = 'horizontal',shrink=0.6)
#cbar2.ax.set_xlabel('number of points',rotation=0)

for index in range(len(df_summary["idx_pair"])):
    #print(df_summary.iloc[index][["idx_pair","idx_obj1","idx_obj2"]])
    text = "{:}({:},{:})".format( int(df_summary.iloc[index]["idx_pair"]), int(df_summary.iloc[index]["idx_obj1"]),int(df_summary.iloc[index]["idx_obj2"]))

    x = df_summary.iloc[index]["muclip"]
    y = df_summary.iloc[index]["sigmaclip"]
    
    if index%2 == 0:
        ax1.text(x-0.001,y+.002,text,size= 'large',horizontalalignment='center',verticalalignment='center')
        ax2.text(x-0.001,y+.002,text,size= 'large',horizontalalignment='center',verticalalignment='center')
        ax3.text(x-0.001,y+.002,text,size= 'large',horizontalalignment='center',verticalalignment='center')
    else:
        ax1.text(x+0.001,y-0.002,text,size= 'large',horizontalalignment='center',verticalalignment='center')
        ax2.text(x+0.001,y-0.002,text,size= 'large',horizontalalignment='center',verticalalignment='center')
        ax3.text(x+0.001,y-0.002,text,size= 'large',horizontalalignment='center',verticalalignment='center')

plt.suptitle(suptitle)
plt.tight_layout()
plt.show()


In [ ]:
fig, ax1 = plt.subplots(1,1,figsize=(18,10))
title = f"AuxtelLightCurves"


sc1=df_summary.plot.scatter(x= "magmax",y="nclip",ax=ax1,s=80, c= "sigmaclip",cmap="jet", label="$|(Z-Y)_1 - (Y-Z)_2|$", grid=True,colorbar=True)
ax1.set_title("dispersion on color $|(Z-Y)_1 - (Y-Z)_2|$ vs pair max(Mag) and nb of points")
ax1.set_xlabel("Max magnitude of the star pair in (Z or Y) (mag)")
ax1.set_ylabel("Number of points on color light curve")


f = plt.gcf()
cax = f.get_axes()[1]
#and we can modify it, i.e.:
cax.set_ylabel('$\sigma(|(Z-Y)_1 - (Z-Y)_2|$ (mag)')

In [ ]:
fig, ax1 = plt.subplots(1,1,figsize=(18,16))
title = f"AuxtelLightCurves"


sc1=df_summary.plot.scatter(x= "magmax",y="nclip",ax=ax1,s=80, c= "sigmaclip",cmap="jet", label="$|(Z-Y)_1 - (Y-Z)_2|$", grid=True,colorbar=True)
ax1.set_title("dispersion on color $|(Z-Y)_1 - (Y-Z)_2|$ vs pair max(Mag) and nb of points")
ax1.set_xlabel("Max magnitude of the star pair in (Z or Y) (mag)")
ax1.set_ylabel("Number of points on color light curve")


f = plt.gcf()
cax = f.get_axes()[1]
#and we can modify it, i.e.:
cax.set_ylabel('$\sigma(|(Z-Y)_1 - (Y-Z)_2|$ (mag)')
for index in range(len(df_summary["idx_pair"])):
    #print(df_summary.iloc[index][["idx_pair","idx_obj1","idx_obj2"]])
    text = "{:}({:},{:})".format( int(df_summary.iloc[index]["idx_pair"]), int(df_summary.iloc[index]["idx_obj1"]),int(df_summary.iloc[index]["idx_obj2"]))

    x = df_summary.iloc[index]["magmax"]
    y = df_summary.iloc[index]["nclip"]
    
    if index%2 == 0:
        ax1.text(x-0.1,y+2,text,size= 'large',horizontalalignment='center',verticalalignment='center')
    else:
        ax1.text(x+0.1,y-2,text,size= 'large',horizontalalignment='center',verticalalignment='center')

## Mapping of object to SED

In [ ]:
file_mappobjtoSED = pd.read_csv("catalog_match_objtopickles_ZY.csv",index_col=0)

In [ ]:
file_mappobjtoSED.set_index("objnum",inplace=True)
file_mappobjtoSED.head()

## Select good pairs

In [ ]:
#df_summary_sel = df_summary.assign(select = lambda row : (row["magmax"]<14.5) & (row["nclip"]>20) & (row["refcol_ap_md"]>0.05))
#df_summary_sel = df_summary.assign(select = lambda row : (row["magmax"]<14.5) & (row["nclip"]>20) )
#df_summary_sel = df_summary.assign(select = lambda row : (row["magmax"]<14.5) & (row["nclip"]>20) & (row["refcol_ap_md"]>0.2) )
#df_summary_sel = df_summary.assign(select = lambda row : (row["magmax"]<14.5) & (row["nclip"]>20) & (row["refcol_ap_md"]>0.2))
df_summary_sel = df_summary.assign(select = lambda row : (row["magmax"]<14.5) & (row["nclip"]>20) & (row["refcol_ap_md"]>0.15))

In [ ]:
#df_summary.assign(select = lambda row : (row["magmax"]<14.5) & (row["nclip"]>20) & (row["refcol_ap_md"]>0.25) )

In [ ]:
df_summary_sel

In [ ]:
good_indexes = df_summary_sel[df_summary_sel["select"]]["idx_pair"].values.astype(int)
print("good_indexes in pair_num ",good_indexes)
Ngood = len(good_indexes)

In [ ]:
df_summary["selection_good"] = df_summary.apply(lambda row : int(row["idx_pair"]) in good_indexes ,axis=1)

In [ ]:
df_summary_sel = df_summary[df_summary["selection_good"]]

In [ ]:
fig,(ax1,ax2) = plt.subplots(1,2,figsize=(12,4))
df_summary["refcol_ap_md"].hist(bins=100,range=(0,0.5),ax=ax1,facecolor="b")
df_summary_sel["refcol_ap_md"].hist(bins=100,range=(0,0.5),ax=ax1,facecolor="r")
ax1.set_xlabel("refcol_ap_md (mag)")

df_summary["refcol_psf_md"].hist(bins=100,range=(0,0.5),ax=ax2,facecolor="b")
df_summary_sel["refcol_psf_md"].hist(bins=100,range=(0,0.5),ax=ax2,facecolor="r")
ax2.set_xlabel("refcol_psf_md (mag)")


plt.suptitle("relative pair color difference")
plt.tight_layout()
plt.show()

## Check if object is known in SED list

In [ ]:
list_of_obj = list(file_mappobjtoSED.index)

In [ ]:
print("list_of_obj :: ",list_of_obj)

In [ ]:
# light curves

all_color_curves = [] # curves color differences vs pav x airmass
all_color_curves1 = []
all_color_curves2 = []
all_seds_pairs = [] # rubin_sim SED pair
all_seds_names = [] # spectral type of the pair
all_sedpairs_rows = []

# loop on pairs
for idx_pair in good_indexes:
    row_pair = df_summary[df_summary.idx_pair == idx_pair].iloc[0]
    
    # find the object numbers of the pair
    idx_obj1 = row_pair["idx_obj1"]
    idx_obj2 = row_pair["idx_obj2"]  
    if idx_obj1 in list_of_obj:
        row1 = file_mappobjtoSED.loc[idx_obj1]
        #print("\t \t obj1",idx_obj1,row1)                             
        fn_sed1 = file_mappobjtoSED.loc[idx_obj1]["sedfnname"]
        sp1 = file_mappobjtoSED.loc[idx_obj1]["sedtype"]
    else:
        fn_sed1 = None
        
    if idx_obj2 in list_of_obj:
        row2 = file_mappobjtoSED.loc[idx_obj2]
        #print("\t \t obj2",idx_obj2,row2)     
        fn_sed2 = file_mappobjtoSED.loc[idx_obj2]["sedfnname"]
        sp2 = file_mappobjtoSED.loc[idx_obj2]["sedtype"]
    else:
        fn_sed2 = None
        
    print(f"- pair {idx_pair} :: \t obj1 = {idx_obj1} --> sed {fn_sed1} ({sp1}), \t obj2 = {idx_obj2} --> sed {fn_sed2} ({sp2})") 
    
    if fn_sed1 != None:
        mag1_bandz= row1["z_ap"]
        sed1 = getSEDfromPickle(fn_sed1,sp1,pc,refcalimag=mag1_bandz,reffilt="z")
        
    if fn_sed2 != None:
        mag2_bandz= row2["z_ap"]
        sed2 = getSEDfromPickle(fn_sed2,sp2,pc,refcalimag=mag2_bandz,reffilt="z")
        
    all_seds_pairs.append([sed1,sed2])
    all_seds_names.append([sp1,sp2])
    all_sedpairs_rows.append([row1,row2])
        
    #z =1
    all_df_magcol  = ComputeColors(pc,sed1,sed2)
    df_diffcol = ComputeRelativeDiffColor(all_df_magcol)
    all_color_curves.append(df_diffcol)
    
    # z= 1.2
    all_df_magcol  = ComputeColors(pc1,sed1,sed2)
    df_diffcol = ComputeRelativeDiffColor(all_df_magcol)
    all_color_curves1.append(df_diffcol)
    
    # z= 2.0
    all_df_magcol  = ComputeColors(pc2,sed1,sed2)
    df_diffcol = ComputeRelativeDiffColor(all_df_magcol)
    all_color_curves2.append(df_diffcol)
    


In [ ]:
len(all_color_curves) 

## Check 

In [ ]:
row_pair["refcol_psf_md"] 

In [ ]:
row_pair["refcol_ap_md"] 

In [ ]:
all_sedpairs_rows[0] 

## Load the model for color correction

In [ ]:
nx = len(all_pwv_colordiffmodel)
ny = len(all_zycolordiff_colordiffmodel)
xg, yg = np.meshgrid(all_pwv_colordiffmodel, all_zycolordiff_colordiffmodel , indexing='ij')

In [ ]:
fig,(ax1,ax2,ax3) = plt.subplots(3,1,figsize=(8,10))
im = ax1.imshow(datagrid_colordiffmodel,origin="lower",cmap="jet",
           extent=(all_pwv_colordiffmodel.min(),all_pwv_colordiffmodel.max(),all_zycolordiff_colordiffmodel.min(),all_zycolordiff_colordiffmodel.max()),aspect='auto')
#cbar

ax1.set_title("relative color difference : $(Z-Y)_1 - (Z-Y)_2$ ")
ax1.set_ylabel("$(Z-Y)_1-(Z-Y)_2$")
ax1.set_xlabel("pwv (mm)")
#ax1.set_aspect('auto')

ax2.plot(all_pwv_colordiffmodel, datagrid_colordiffmodel[0,:],'b-')
ax2.set_xlabel("pwv (mm)")
ax2.set_ylabel("rel. col. : $(Z-Y)_1 - (Z-Y)_2$")
ax3.plot(all_pwv_colordiffmodel, datagrid_colordiffmodel[-1,:],'r-')
ax3.set_xlabel("pwv (mm)")
ax3.set_ylabel("rel. col. : $(Z-Y)_1 - (Z-Y)_2$")

plt.tight_layout()
plt.show()

In [ ]:
interp_colordiffmodel = RegularGridInterpolator((all_pwv_colordiffmodel, all_zycolordiff_colordiffmodel), datagrid_colordiffmodel.T, bounds_error=False, fill_value=None)

In [ ]:
# wavelength bin colors
jet = plt.get_cmap('jet')
cNorm = mpl.colors.Normalize(vmin=all_zycolordiff_colordiffmodel.min(), vmax=all_zycolordiff_colordiffmodel.max())
scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(10,6))

for z_y0 in all_zycolordiff_colordiffmodel:
    the_color = scalarMap.to_rgba(z_y0, alpha=1)
    
    pts = list(zip(all_pwv_colordiffmodel,z_y0*np.ones(len(all_pwv_colordiffmodel))))  
    c = interp_colordiffmodel(pts)
    ax.plot(all_pwv_colordiffmodel,c,'-',c=the_color)
    
cbar = fig.colorbar(scalarMap,ax=ax, orientation='vertical', label='Z-Y(pwv=0) (mag)')    
ax.set_xlabel("pwv (mm)")    
#ax.set_ylabel("Czy1_Czy2_abmag_rel (mmag)")
ax.set_ylabel("rel. col. : $\delta_{12}C_{zy} $ (mmag)")
ax.set_title("Relative color difference Z-Y from star1,star2")

### Check the seds and the simulations

In [ ]:
# wavelength bin colors
jet = plt.get_cmap('jet')
cNorm = mpl.colors.Normalize(vmin=-.26, vmax=.26)
scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)

In [ ]:
photom_wl

In [ ]:
F0

In [ ]:
# loop on pairs
for idx,idx_pair in enumerate(good_indexes):
    row_pair = df_summary[df_summary.idx_pair == idx_pair].iloc[0]
    print(f"pair index = {row_pair}")
    # find the object numbers of the pair
    idx_obj1 = int(row_pair["idx_obj1"])
    idx_obj2 = int(row_pair["idx_obj2"])  
    sed1 = all_seds_pairs[idx][0] 
    sed2 = all_seds_pairs[idx][1] 
    sedn1 = all_seds_names[idx][0] 
    sedn2 = all_seds_names[idx][1] 
    row1 = all_sedpairs_rows[idx][0] 
    row2 = all_sedpairs_rows[idx][1] 
    
    # colors for data
    refcol_psf_md = row_pair["refcol_psf_md"]
    refcol_ap_md = row_pair["refcol_ap_md"]
    
    mags1 = np.array([row1["g_ap"],row1["r_ap"],row1["i_ap"],row1["z_ap"], row1["y_ap"]])
    mags1_err = np.array([row1["sg_ap"],row1["sr_ap"],row1["si_ap"],row1["sz_ap"], row1["sy_ap"]])
    
    mags2 = np.array([row2["g_ap"],row2["r_ap"],row2["i_ap"],row2["z_ap"], row2["y_ap"]])
    mags2_err = np.array([row2["sg_ap"],row2["sr_ap"],row2["si_ap"],row2["sz_ap"], row2["sy_ap"]])
    magz1 = mags1[3]
    magz2 = mags2[3]
    

    
    fig,axs = plt.subplots(1,2,figsize=(12,5))
    ax1,ax2 = axs.flatten()
    ax1.plot(sed1.wavelen,-2.5*np.log10(sed1.fnu/F0),"b-")
    ax1.plot(sed2.wavelen,-2.5*np.log10(sed2.fnu/F0),"r-")
    
    ax1.errorbar(photom_wl,mags1,yerr=mags1_err,fmt=".",color="b",ecolor="k",elinewidth=1.)  
    ax1.scatter(photom_wl,mags1,marker="o",color="b",s=50,label=f"obj1 {idx_obj1} (mag(z) = {magz1:.1f} mag)") 
    
    ax1.errorbar(photom_wl,mags2,yerr=mags2_err,fmt=".",color="r",ecolor="k",elinewidth=1.)  
    ax1.scatter(photom_wl,mags2,marker="o",color="r",s=50,label=f"obj2 {idx_obj2} (mag(z) = {magz2:.1f} mag)") 
    
    ax1.set_ylim(12.,17.)
    ax1.set_xlim(300,1200)
    ax1.yaxis.set_inverted(True)
    ax1.set_ylabel("magnitude (AB-mag)")
    ax1.set_xlabel("$\\lambda$ (nm)")
    
    ax1.set_title(f"Object pair {idx_pair} sed pickles : {sedn1}, {sedn2}")
    ax1.legend()
    ax3 = ax1.twinx()
    for ifilt,f in enumerate(filter_tagnames):
        ax3.fill_between(pc.bandpass_total_std[f].wavelen,pc.bandpass_total_std[f].sb,color=filter_color[ifilt],alpha=0.1) 
    ax3.set_yticks([])
    ax3.set_ylim(0.,0.8)
    
    # get the color curve from the index idx of the selected goodindexes
    the_color_curve = all_color_curves[idx]
    # the color difference calculated from tthe SED
    z_y0 = the_color_curve.loc[0,"Czy1_Czy2_abmag"]
    
    print(f"\t - the  z_y0 = { z_y0:.3f} mag")
    the_color = scalarMap.to_rgba(z_y0, alpha=1)
    ax2.plot(the_color_curve.pwv,the_color_curve.Czy1_Czy2_instmag_rel.abs(),":",c=the_color,label="calc from fitted SEDs")
    
    
    # but with the model we use :
    
   
    the_color2 = scalarMap.to_rgba( refcol_ap_md, alpha=1)
    pts = list(zip(all_pwv_colordiffmodel,refcol_ap_md*np.ones(len(all_pwv_colordiffmodel))))  
    cmodel = np.abs(interp_colordiffmodel(pts))
    ax2.plot(all_pwv_colordiffmodel,cmodel,'-',c=the_color2,label="calc. from measured color diff Z-Y")
    

    ax2.set_xlabel("pwv (mm)")    
    ax2.set_ylabel("$|\delta_{12}C_{zy}|$ (mmag)")
    ax2.set_title(f"Relative color difference (Z-Y) from  pair {idx_pair} ({idx_obj1},{idx_obj2})")
    ax2.set_ylim(0.,20.)
    
    text_str1 = f"meas : refcol_ap_md  = {refcol_ap_md:.3f} mag"
    text_str2 = f"meas : refcol_psf_md = {refcol_psf_md:.3f} mag"
    text_str3 = f"seds : z_y0          = {z_y0:.3f} mag"
    text_str = "\n".join((text_str1,text_str2,text_str3))
    ax2.text(0.1, 0.95, text_str, transform=ax2.transAxes, fontsize=14,verticalalignment='top', bbox=props)
    ax2.legend(loc="lower right")
    ax2.grid()
    
    
    plt.tight_layout()
    plt.show()
    

In [ ]:
# wavelength bin colors
jet = plt.get_cmap('jet')
cNorm = mpl.colors.Normalize(vmin=-.26, vmax=.26)
scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(10,6))

for idx,idx_pair in enumerate(good_indexes):
    the_color_curve = all_color_curves[idx]
    z_y0 = the_color_curve.loc[0,"Czy1_Czy2_abmag"]
    the_color = scalarMap.to_rgba(z_y0, alpha=1)
    ax.plot(the_color_curve.pwv,the_color_curve.Czy1_Czy2_instmag_rel,c=the_color)

cbar = fig.colorbar(scalarMap,ax=ax, orientation='vertical', label='Z-Y(pwv=0) (mag)')    
ax.set_xlabel("pwv (mm)")    
ax.set_ylabel("Czy1_Czy2_abmag_rel (mmag)")
ax.set_title("Relative color difference Z-Y from SEDs (star1,star2)")

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(10,6))

for idx,idx_pair in enumerate(good_indexes):
    the_color_curve = all_color_curves[idx]
    the_color_curve1 = all_color_curves1[idx]
    the_color_curve2 = all_color_curves2[idx]
    z_y0 = the_color_curve2.loc[0,"Czy1_Czy2_abmag"]
    the_color = scalarMap.to_rgba(z_y0, alpha=1)
    ax.plot(the_color_curve2.pwv*2,the_color_curve2.Czy1_Czy2_instmag_rel,c=the_color)
    ax.plot(the_color_curve.pwv*1,the_color_curve.Czy1_Czy2_instmag_rel,":",c=the_color)
    ax.plot(the_color_curve1.pwv*1.2,the_color_curve1.Czy1_Czy2_instmag_rel,":",c=the_color)

cbar = fig.colorbar(scalarMap,ax=ax, orientation='vertical', label='Z-Y(pwv=0) (mag)')    
ax.set_xlabel("pwv(airmass)")    
ax.set_ylabel("Czy1_Czy2_abmag_rel (mmag)")
ax.set_title("Relative color difference Z-Y from (star1,star2)")

## Plot per object light curve

In [ ]:
props = dict(boxstyle='round', facecolor="white", alpha=0.5)
textstr0 = '\n'.join((
    r'$N_{pairs}=%.0f$' % (len(good_indexes), ),
    r'$\theta_{sep}<%.0f$ arcsec' % (sep_max, )))

In [ ]:
# wavelength bin colors
jet = plt.get_cmap('jet')
cNorm = mpl.colors.Normalize(vmin=0, vmax=0.3)
scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)
all_colors = scalarMap.to_rgba(all_objectplotcolor, alpha=1)

In [ ]:
from matplotlib.dates import DateFormatter
#date_form = DateFormatter("%y-%m-%dT%H:%M")
date_form = DateFormatter("%y-%m-%d")
#fig,axs = plt.subplots(2,1,figsize=(14,8),sharex=True)

fig = plt.figure(figsize=(14,10),constrained_layout=True)
gs = GridSpec(2, 1, wspace=0, hspace=0.1, height_ratios=[1,2],figure=fig)

ax1 = fig.add_subplot(gs[0])
ax2 = fig.add_subplot(gs[1],sharex=ax1)


df_spec.plot(x="Time",y="PWV [mm]_x",ax=ax1,marker='+',c="r",lw=0.0,grid=True,label="spectro PWV_x")

all_tmin = []
all_tmax = []

for idx,df_col in enumerate(all_colors_diff):
    the_color = all_colors[idx]

    if idx in good_indexes:
        df_col.plot(x="time1",y="apcol12_s12",ax=ax2,marker="o",c=the_color,lw=0.5,grid=True,legend=False)
        #ax2.errorbar(df_col.time1,df_col.psfcol12_s12,yerr=df_col.psfcol12Err_s12,color=the_color,ecolor="k",fmt="o")
        ax2.errorbar(df_col.time1,df_col.apcol12_s12,yerr=df_col.apcol12Err_s12,color=the_color,ecolor="k",fmt="o")

        df_col.plot(x="Time",y="PWV [mm]_x",ax=ax1,marker="o",c="b",lw=0.,grid=True,label="")
        
        tmin = df_col["time1"].min()
        tmax = df_col["time1"].max()
        dt = tmax-tmin
        tmin = tmin-dt/10
        tmax = tmax+dt/10
        all_tmin.append(tmin)
        all_tmax.append(tmax)


ax2.set_xlabel("time")
ax2.xaxis.set_major_formatter(date_form)

the_tmin=np.min(all_tmin)
the_tmax=np.max(all_tmax)

ax2.set_xlim(the_tmin,the_tmax)
ax2.set_title("Auxtel Aper Photometry: $|(Z-Y)_1-(Z-Y)_2|$ (selected pairs on mag)")
ax1.legend(loc="upper left")
ax1.set_ylabel("PWV (mm)")
ax1.set_title("Auxtel Spectroscopy : precipitable water vapor")
#ax2.legend(loc="upper left")
ax2.set_ylabel("Aperture: $|(Z-Y)_1-(Z-Y)_2|$")
ax2.set_ylim(0.,0.4)
ax2.text(0.01, 0.95, textstr0, transform=ax2.transAxes, fontsize=14,verticalalignment='top', bbox=props)

title = suptitle
plt.suptitle(title,y=1)
plt.tight_layout()

In [ ]:

fig,ax  = plt.subplots(figsize=(6,6))


for idx_pair,df_col in enumerate(all_colors_diff):
    the_color = all_colors[idx_pair]

    if idx_pair in good_indexes:
        refcol_ap_md = (df_summary[df_summary["idx_pair"].astype(int) == idx_pair]["refcol_ap_md"]).values[0]
        #df_col.plot(x="PWV [mm]_x",y="apcol12_s12",ax=ax,marker="o",c=the_color,lw=0.5,grid=True,legend=False)
        #ax2.errorbar(df_col.time1,df_col.psfcol12_s12,yerr=df_col.psfcol12Err_s12,color=the_color,ecolor="k",fmt="o")
        ax.errorbar(df_col["PWV [mm]_x"],df_col.apcol12_s12-refcol_ap_md,yerr=df_col.apcol12Err_s12,color=the_color,ecolor="k",fmt="o")
        
ax.grid()       
title = suptitle
plt.suptitle(title,y=1)
plt.tight_layout()

## Loop to plot relative light curves one by one and DO straight line fitting

In [ ]:
from scipy.optimize import curve_fit,least_squares

def func_residuals_colorterm(params,*args):
        """
    
        arg = vector of info to do the fit
        params : parameeters to fit
    
        """

        # decode parameters
        colorshift = params[0]
        

        # decode data to fit
        xdata = args[0]    
        ydata = args[1]    
        yerrdata = args[2]
  
        # decode what is usefull for model
        pwvairmass_m = args[3]    
        colorterm_m = args[4]
        
        interp_model = np.interp(xdata, pwvairmass_m, colorterm_m)
    
        residuals = (ydata +colorshift - interp_model)/yerrdata
        return residuals     


def func_residuals_nocolorterm(params,*args):
        """
    
        arg = vector of info to do the fit
        params : parameeters to fit
    
        """

        # decode parameters
        colorshift = params[0]
        

        # decode data to fit
        xdata = args[0]    
        ydata = args[1]    
        yerrdata = args[2]
  
    
        residuals = (ydata +colorshift)/yerrdata
        return residuals        
    
def func_residuals_linearcolorterm(params,*args):
        """
        
        arg = vector of info to do the fit
        params : parameeters to fit
        
        """
    
        # decode parameters
        colorshift = params[0]
        colorslope = params[1]
        

        # decode data to fit
        xdata = args[0]    
        ydata = args[1]    
        yerrdata = args[2]
  
    
        residuals = (ydata +colorshift - xdata*colorslope)/yerrdata
        return residuals   
    
def func_residuals_powerlawcolorterm(params,*args):
        """
        
        arg = vector of info to do the fit
        params : parameeters to fit
        
        """
    
               # decode parameters
        colorshift = params[0]
        K = params[1]
        

        # decode data to fit
        xdata = args[0]    
        ydata = args[1]    
        yerrdata = args[2]
  
    
        residuals = (ydata +colorshift - K*np.power(xdata,0.773))/yerrdata
        return residuals 
    

class FitAtmosphericColorParams:
    """
    Class to handle a buch of different methods for fitting
    """
    def __init__(self):
        print("Init FitAtmosphericColorParams")
        
    
        

        
    def fit_colorterm(self,params0,xdata,ydata,yerrdata,pwvairmass,colortermmodel):
        """
        Fit the color term

        """
            
        res_fit = least_squares(func_residuals_colorterm, params0,bounds=([-100.],[100.]),args=[xdata,ydata,yerrdata,pwvairmass,colortermmodel])
        
        #alpha_fit,pwv_fit,oz_fit = res_fit.x
    
    
        # results fo the fit
        popt= res_fit.x
        J = res_fit.jac
        pcov = np.linalg.inv(J.T.dot(J))
        sigmas = np.sqrt(np.diagonal(pcov))
        cost=res_fit.cost
        residuals = res_fit.fun
        chi2=np.sum(residuals**2)
        ndeg = J.shape[0]- popt.shape[0]
        chi2_per_deg = chi2/ndeg
        #pwve = sigmas[1]*np.sqrt(chi2_per_deg)
        #oze =sigmas[2]*np.sqrt(chi2_per_deg)
        #greye = sigmas[0]*np.sqrt(chi2_per_deg)
        
        fit_dict = {"chi2":chi2,"ndeg":ndeg,"chi2_per_deg":chi2_per_deg,"popt":popt,"sigmas":sigmas}

        return res_fit,fit_dict
    
    
    def fit_nocolorterm(self,params0,xdata,ydata,yerrdata):
        """
        Fit No color term : constant

        """
            
        res_fit = least_squares(func_residuals_nocolorterm, params0,bounds=([-100.],[100.]),args=[xdata,ydata,yerrdata])
        
        #alpha_fit,pwv_fit,oz_fit = res_fit.x
    
    
        # results fo the fit
        popt= res_fit.x
        J = res_fit.jac
        pcov = np.linalg.inv(J.T.dot(J))
        sigmas = np.sqrt(np.diagonal(pcov))
        cost=res_fit.cost
        residuals = res_fit.fun
        chi2=np.sum(residuals**2)
        ndeg = J.shape[0]- popt.shape[0]
        chi2_per_deg = chi2/ndeg
        #pwve = sigmas[1]*np.sqrt(chi2_per_deg)
        #oze =sigmas[2]*np.sqrt(chi2_per_deg)
        #greye = sigmas[0]*np.sqrt(chi2_per_deg)
        
        fit_dict = {"chi2":chi2,"ndeg":ndeg,"chi2_per_deg":chi2_per_deg,"popt":popt,"sigmas":sigmas}

        return res_fit,fit_dict    
    
    def fit_linearcolorterm(self,params0,xdata,ydata,yerrdata):
        res_fit = least_squares(func_residuals_linearcolorterm, params0,bounds=([-100.,-50.],[100.,50.]),args=[xdata,ydata,yerrdata])
        
        #alpha_fit,pwv_fit,oz_fit = res_fit.x
    
    
        # results fo the fit
        popt= res_fit.x
        J = res_fit.jac
        pcov = np.linalg.inv(J.T.dot(J))
        sigmas = np.sqrt(np.diagonal(pcov))
        cost=res_fit.cost
        residuals = res_fit.fun
        chi2=np.sum(residuals**2)
        ndeg = J.shape[0]- popt.shape[0]
        chi2_per_deg = chi2/ndeg
        #pwve = sigmas[1]*np.sqrt(chi2_per_deg)
        #oze =sigmas[2]*np.sqrt(chi2_per_deg)
        #greye = sigmas[0]*np.sqrt(chi2_per_deg)
        
        fit_dict = {"chi2":chi2,"ndeg":ndeg,"chi2_per_deg":chi2_per_deg,"popt":popt,"sigmas":sigmas}

        return res_fit,fit_dict    
                     
                     
    def fit_powerlawcolorterm(self,params0,xdata,ydata,yerrdata):
        res_fit = least_squares(func_residuals_powerlawcolorterm, params0,bounds=([-100.,-50.],[100.,50.]),args=[xdata,ydata,yerrdata])
        
        #alpha_fit,pwv_fit,oz_fit = res_fit.x
    
    
        # results fo the fit
        popt= res_fit.x
        J = res_fit.jac
        pcov = np.linalg.inv(J.T.dot(J))
        sigmas = np.sqrt(np.diagonal(pcov))
        cost=res_fit.cost
        residuals = res_fit.fun
        chi2=np.sum(residuals**2)
        ndeg = J.shape[0]- popt.shape[0]
        chi2_per_deg = chi2/ndeg
        #pwve = sigmas[1]*np.sqrt(chi2_per_deg)
        #oze =sigmas[2]*np.sqrt(chi2_per_deg)
        #greye = sigmas[0]*np.sqrt(chi2_per_deg)
        
        fit_dict = {"chi2":chi2,"ndeg":ndeg,"chi2_per_deg":chi2_per_deg,"popt":popt,"sigmas":sigmas}
        return res_fit,fit_dict    


In [ ]:
DTMAX = 3600
#DTMAX = 4800.
CHI2FITMAX = 50
props = dict(boxstyle='round', facecolor="white", alpha=0.1)

In [ ]:
FLAG_MODEL_FIT = True
yylabel = "$|\delta_{12}C_{zy}|\; (mmag)$"
xxlabel = "$PWV$ (mm) in direction obs"
fitter = FitAtmosphericColorParams()

# to find dependence
all_cslope = []
all_ck = []

all_cslope_err = []
all_ck_err = []

all_d1dCzy0 = []

if FLAG_MODEL_FIT:
    
    df_fit_ap = pd.DataFrame(columns=["idx_pair","idx_summary","color","slope","slope_err","sep"]) 
    date_form = DateFormatter("%y-%m-%d")

    count = 0
    #for good_idx in good_indexes:
    for idx,good_idx in enumerate(good_indexes):
        
        
        # get the color now
        df_col = all_colors_diff[good_idx]  
        #Extract the row in df_summary with the pair number good_idx
        df_summary_row = df_summary[df_summary["idx_pair"].astype(int) == good_idx]
        
        
        print("===================================================================================")
        print(df_summary_row )
        #print(df_col) Too long
        
        # extract the corresponding index and refcol_ap_md column
        index_in_summary = df_summary_row.index[0]
        refcol_ap_md = df_summary_row["refcol_ap_md"].values[0]
        print(f"\t ==> refcol_ap_md = {refcol_ap_md:.3f} mag")
        
        
   
        
        # compute a recentered colors fro plotting
        df_col["relapcol"] = df_col.apcol12_s12-refcol_ap_md

        separation_angle = df_summary.loc[index_in_summary,"sep"]
        
        
        # get the light curve
        PWV = df_col["PWV [mm]_x"].values 
        X =  df_col["Time"].values
        Y =  (df_col["PWV [mm]_x"] *  df_col["airmassb1"]).values
        Yerr = (df_col["PWV [mm]_err_x"]*df_col["airmassb1"]).values
        DT = df_col["dt_abs_specphot"].values # time difference
        CHI2 = df_col["CHI2_FIT"].values
        
        #goods = (~np.isnat(X)) & (DT < DTMAX) & (np.isnan(CHI2)) & (CHI2<CHI2FITMAX)
        goods = (~np.isnat(X)) & (DT < DTMAX)  & (CHI2<CHI2FITMAX) & (PWV>0.1) & (PWV<9.9)
        
        X=X[goods]
        Y=Y[goods]
        Yerr = Yerr[goods]
        DT = DT[goods]
        
      
        
        # the color diff light curve where all usefull variable are selected
        df_col_subset = df_col[["Time","PWV [mm]_x","PWV [mm]_err_x","dt_abs_specphot","airmassb1","relapcol","apcol12Err_s12","CHI2_FIT"]][goods]
        df_col_subset["pwvxairmass"] = df_col_subset["PWV [mm]_x"]*df_col_subset["airmassb1"]
        
        #print(df_col_subset,len(df_col_subset))
        
        #print("X",len(X))
        #print("Y",len(Y))
        #print("Yerr",len(Yerr))
        #print("DT",len(DT))
        
        tmin = df_col_subset["Time"].min()
        tmax = df_col_subset["Time"].max()
        dt = tmax-tmin
        tmin = tmin-dt/10
        tmax = tmax+dt/10
        the_tmin=tmin
        the_tmax=tmax
        
        #DT bin colors
        jet = plt.get_cmap('jet')
        cNorm = mpl.colors.Normalize(vmin=DT.min(), vmax=DT.max())
        scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)
        all_colors_dt = scalarMap.to_rgba(DT, alpha=1)
        
        if len(X)<5:
            print(f"{idx}) pair {good_idx} rejected too small number of coincidence photom/spectro ")
            continue
        
        # Not regected    
        all_d1dCzy0.append(refcol_ap_md)
        
        
        # the SED model not used anymore    
        the_color_curve = all_color_curves[idx]
        the_color_curve1 = all_color_curves1[idx]
        the_color_curve2 = all_color_curves2[idx]

        # decomposition of the figure in 4 subplots
        # =========================================
        fig = plt.figure(figsize=(14,18),constrained_layout=True)
        gs = GridSpec(4, 2, wspace=0, hspace=0.01, height_ratios=[1,1,1,1],figure=fig)

        ax1 = fig.add_subplot(gs[0,0])
        ax2 = fig.add_subplot(gs[1,0])
        ax3 = fig.add_subplot(gs[0,1])
        ax4 = fig.add_subplot(gs[1,1])
        ax5 = fig.add_subplot(gs[2,0])
        ax6 = fig.add_subplot(gs[2,1])
        ax7 = fig.add_subplot(gs[3,0])
        ax8 = fig.add_subplot(gs[3,1])
        

        # subplot1: measurement of pwv in Auxtel
        # =========================================
        #df_spec.plot(x="Time",y="PWV [mm]_x",ax=ax1,marker='+',c="r",lw=0.0,grid=True,label="all PWV meas by spectro",alpha=0.1)


        # compute the string
        textstr = '\n'.join((
        "(Z-Y) diff star pair :"
        r'objects =(%.0f,%.0f)' % (df_summary.loc[index_in_summary,"idx_obj1"],df_summary.loc[index_in_summary,"idx_obj2"],  ),
        r'$\theta_{sep} = %.0f$ arcsec' % (df_summary.loc[index_in_summary,"sep"], ),
        r'$|(Z-Y)_1- (Z-Y)_2| \simeq%.3f$  mag' % (df_summary.loc[index_in_summary,"refcol_ap_md"], )))

        title = "pair {} :: obj1= {} obj2 = {} ".format(good_idx,df_summary.loc[index_in_summary,"idx_obj1"].astype(int),
                                                    df_summary.loc[index_in_summary,"idx_obj2"].astype(int))
        
        
        
        
        ax1.errorbar(X,Y,yerr=Yerr,color=the_color,ecolor="k",fmt=".")
        #ax1.scatter(X,Y,c=the_color,marker= "o",s=50)
       
        df_col_subset.plot.scatter(x="Time",y="pwvxairmass",ax=ax1,marker="o",c="dt_abs_specphot",s=50,colormap="jet")
        
        ax1.set_title(title)
        ax1.set_ylabel("PWV(airmass) (mm) (meas spectro)")
        ax1.xaxis.set_major_formatter(date_form)
        ax1.grid()
        ax1.set_xticks(ax1.get_xticks(), ax1.get_xticklabels(), rotation=45, ha='right')
        ax1.set_xlim(tmin,tmax)
   
    
        # subplot2 : the ligth curve
        # ==========================
        ax2.errorbar(df_col.time1,df_col.apcol12_s12-refcol_ap_md,yerr=df_col.apcol12Err_s12,color=the_color,ecolor="k",fmt="o")
        
        #ax2.text(0.01, 0.97, textstr, transform=ax2.transAxes, fontsize=10,verticalalignment='top', bbox=props)
        ax2.set_ylabel("$\delta_{12}C_{zy}$ (mag)")
        ax2.set_xlabel("time")
        ax2.grid()
        ax2.set_xticks(ax2.get_xticks(), ax2.get_xticklabels(), rotation=45, ha='right')
        ax2.xaxis.set_major_formatter(date_form)
        ax2.set_xlim(tmin,tmax)
    
    
        #subplot3 : the attenuation vs pwv x airmass
        # ============================================
        X =  df_col_subset["pwvxairmass"].values
        Y =  df_col_subset["relapcol"].values*1000.
        Xerr = (df_col_subset["PWV [mm]_err_x"]*df_col_subset["airmassb1"]).values
        Yerr = df_col_subset.apcol12Err_s12.values*1000.
        
        
        # remove nan in X
        isgood = ~np.isnan(X)
        X = X[isgood]
        Y = Y[isgood]
             
        Xerr = Xerr[isgood]
        Yerr = Yerr[isgood]
        
      
    
        #ax3.errorbar(X,Y,xerr=Xerr,yerr=Yerr,color=the_color,ecolor="k",fmt=".")
        #ax3.scatter(X,Y,c=the_color,marker= "o",s=50)
        #ax3.grid()
    
        #ax3.set_xlabel("PWV x airmass (mm)")
        #ax3.set_ylabel("relative apert col diff (mag)")
        #ax3.set_title("fit line aperture color diff vs PWV")
        
        
       
        
        
        #suppress the sed model
        #ax3.plot(the_color_curve.pwv*1,np.abs(the_color_curve.Czy1_Czy2_instmag_rel),"-",lw=3,c="r",label="color term")
        #ax3.plot(the_color_curve1.pwv*1.2,np.abs(the_color_curve1.Czy1_Czy2_instmag_rel)/1000,":",c=the_color)
    
        #ax4.errorbar(df_col["PWV [mm]_x"],df_col.relapcol,xerr=df_col["PWV [mm]_err_x"],yerr=df_col.apcol12Err_s12,color=the_color,ecolor="k",fmt="o")
        #ax4.grid()
        #ax4.set_xlabel("PWV [mm]_x (spectro)")
        
        params0 = [0.] 
        
        ## old way to calculate with sed pickles fitted
        Xmodel = the_color_curve.pwv.values*1.
        Ymodel = np.abs(the_color_curve.Czy1_Czy2_instmag_rel.values)
        
        # fit with diffcol emulator
        pts = list(zip(all_pwv_colordiffmodel,refcol_ap_md*np.ones(len(all_pwv_colordiffmodel))))  
        cmodel = np.abs(interp_colordiffmodel(pts))
        Xmodel = all_pwv_colordiffmodel
        Ymodel = np.abs(cmodel)
        ax3.plot(Xmodel,Ymodel ,"-",lw=3,c="r",label="color term")
        
        
        
        res_fit,fit_dict = fitter.fit_colorterm(params0,
                                      X,
                                      Y,
                                      Yerr,
                                      Xmodel,
                                      Ymodel)
    
        cs = res_fit.x[0]
        chi2ndeg = fit_dict["chi2_per_deg"]
        
        ax3.errorbar(X,Y+cs,xerr=Xerr,yerr=Yerr,ecolor="k",fmt=".",label="data")
        df_col_subset["refcolshift_1"] = Y+cs
        df_col_subset.plot.scatter(x="pwvxairmass",y="refcolshift_1",ax=ax3,marker="o",c="dt_abs_specphot",s=50,colormap="jet")
        
        
        ax3.scatter(X,Y+cs,marker= "o",color=all_colors_dt,s=50)
        ax3.legend(loc="upper left")
        ax3.grid()
        
        textstr3 = '\n'.join((textstr,"$\chi^2/ndf = $" + f"{chi2ndeg:.1f}"))
        ax3.text(0.05, 0.25, textstr3, transform=ax3.transAxes, fontsize=10,verticalalignment='top', bbox=props)
        #ax3.set_xlabel("pwv (mm) in dir obs")
        #ax3.set_ylabel("relative apert col diff (mmag)")
        
        ax3.set_xlabel(xxlabel)
        ax3.set_ylabel(yylabel)
        ax3.set_title("apert phot : fit full model (1 param)")
        
        
        #ax3.grid()
        
        # subplot 4 : No plot
        #===================
        # ax4.set_axis_off()
    
    
        res_fit,fit_dict = fitter.fit_nocolorterm(params0,X,Y,Yerr)
                                     
        cs = res_fit.x[0]
        chi2ndeg = fit_dict["chi2_per_deg"]
        
        ax4.errorbar(X,Y+cs,xerr=Xerr,yerr=Yerr,ecolor="k",fmt=".",label="data")
        ax4.scatter(X,Y+cs,c="b",marker= "o",s=50)
        df_col_subset["refcolshift_2"] = Y+cs
        df_col_subset.plot.scatter(x="pwvxairmass",y="refcolshift_2",ax=ax4,marker="o",c="dt_abs_specphot",s=50,colormap="jet")
        
        #ax4.plot(ax4.get_xlim(),[cs,cs],"r-",label="no color model")
        ax4.plot(ax4.get_xlim(),[0,0],"r-",label="no PWV dependence")
        ax4.legend(loc="upper left") 
        ax4.grid()
        
        textstr4 = '\n'.join((textstr,"$\chi^2/ndf = $" + f"{chi2ndeg:.1f}"))
        ax4.text(0.05, 0.25, textstr4, transform=ax4.transAxes, fontsize=10,verticalalignment='top', bbox=props)
        #ax4.set_xlabel("pwv (mm) in dir obs")
        #ax4.set_ylabel("relative apert col diff (mmag)")
        
        ax4.set_xlabel(xxlabel)
        ax4.set_ylabel(yylabel)
        ax4.set_title("apert phot : fit a constant (1 param)")    
    
        
        # subplot 5 : No plot
        #===================
        #ax5.set_axis_off()
        df_col["td_rel_specphot"].hist(bins=25,range=(-DTMAX ,DTMAX),histtype="step",color="r",lw=2,ax=ax5)
        ax5.set_xlabel("$\Delta t (spectro-photom)$ (s)")
        
        
        
        # subplot 6 : Linear
        #===================
        # ax4.set_axis_off()
    
        params0 = [0.,0.]
    
        res_fit,fit_dict = fitter.fit_linearcolorterm(params0,X,Y,Yerr)
                                     
        cs = res_fit.x[0]
        cslope = res_fit.x[1]
        cslope_err = fit_dict["sigmas"][1]
        
        
        chi2ndeg = fit_dict["chi2_per_deg"]
        
        ax6.errorbar(X,Y+cs,xerr=Xerr,yerr=Yerr,ecolor="k",fmt=".",label="data")
        ax6.scatter(X,Y+cs,c="b",marker= "o",s=50)
        df_col_subset["refcolshift_3"] = Y+cs
        df_col_subset.plot.scatter(x="pwvxairmass",y="refcolshift_3",ax=ax6,marker="o",c="dt_abs_specphot",s=50,colormap="jet")
        
        xfit = np.linspace(0.,15.,50)
        yfit = cslope*xfit
        
        ax6.plot(xfit,yfit,"r-",label="linear color model")
        ax6.legend(loc="upper left") 
        ax6.grid()
        
        textstr6 = '\n'.join((textstr,"$\chi^2/ndf = $" + f"{chi2ndeg:.1f}",f"slope = {cslope:.3f} mmag/mm"))
        ax6.text(0.05, 0.3, textstr6, transform=ax6.transAxes, fontsize=10,verticalalignment='top', bbox=props)
        #ax6.set_xlabel("pwv (mm) in dir obs")
        #ax6.set_ylabel("relative apert col diff (mmag)")
        
        ax6.set_xlabel(xxlabel)
        ax6.set_ylabel(yylabel)
        ax6.set_title("apert phot : fit a line (1 param)")    
        
        all_cslope.append(cslope)
        all_cslope_err.append(cslope_err)
 
        
        # subplot 7 : Linear
        #===================
        ax7.set_axis_off()
        
        
        
        # subplot 8 : Linear
        #===================
        # ax4.set_axis_off()
    
        params0 = [0.,1.]
    
        res_fit,fit_dict = fitter.fit_powerlawcolorterm(params0,X,Y,Yerr)
                                     
        cs = res_fit.x[0]
        K = res_fit.x[1]
        K_err =  fit_dict["sigmas"][1]
        
        chi2ndeg = fit_dict["chi2_per_deg"]
        
        ax8.errorbar(X,Y+cs,xerr=Xerr,yerr=Yerr,ecolor="k",fmt=".",label="data")
        ax8.scatter(X,Y+cs,c="b",marker= "o",s=50)
        df_col_subset["refcolshift_4"] = Y+cs
        df_col_subset.plot.scatter(x="pwvxairmass",y="refcolshift_4",ax=ax8,marker="o",c="dt_abs_specphot",s=50,colormap="jet")
        
        xfit = np.linspace(0.,15.,50)
        yfit = K*np.power(xfit,0.773)
        
        ax8.plot(xfit,yfit,"r-",label="fit : $K.PWV^\\beta$")
        ax8.legend(loc="upper left") 
        ax8.grid()
        
        textstr8 = '\n'.join((textstr,"$\chi^2/ndf = $" + f"{chi2ndeg:.1f}",f"K = {K:.3f} mmag"))
        ax8.text(0.05, 0.3, textstr8, transform=ax8.transAxes, fontsize=10,verticalalignment='top', bbox=props)
        ax8.set_xlabel("pwv (mm) in dir obs")
        ax8.set_ylabel("relative apert col diff (mmag)")
        
        
        ax8.set_xlabel(xxlabel)
        ax8.set_ylabel(yylabel)
        ax8.set_title("apert phot : fit $K.PWV^\\beta$  (2-param, $\\beta$ = 0.773)")    
        
        all_ck.append(K)
        all_ck_err.append(K_err)
       
        
        count += 1
        #plt.tight_layout()
        plt.show()
        
        
        
        #if count > 2:
        #    break
   


In [ ]:
len(all_d1dCzy0)

In [ ]:
len(all_cslope)

In [ ]:
fig,(ax1,ax2) = plt.subplots(1,2,figsize=(12,4))
ax1.scatter(all_d1dCzy0 ,all_cslope,marker="o",c='b')
ax1.errorbar(all_d1dCzy0 ,all_cslope,yerr= all_cslope_err,fmt=".",c='b',ecolor="k")
ax1.grid()
ax1.set_xlabel("$|\Delta_{12} C_{zy}|$ (mag)")
ax1.set_ylabel("slope (mmag/mm)")
ax1.set_title("from linear fit")
ax1.axhline(0,c='k',ls="-.")
ax1.axvspan(0.15, 0.2, alpha=0.5, color='grey')


ax2.scatter(all_d1dCzy0 ,all_ck,marker="o",c='b')
ax2.errorbar(all_d1dCzy0 ,all_ck,yerr= all_ck_err,fmt=".",c='b',ecolor="k")
ax2.grid()
ax2.set_xlabel("$|\Delta_{12} C_{zy}|$ (mag)")
ax2.set_ylabel("K (mmag)")
ax2.set_title("from model fit : $|\delta_{12}C_{zy}| \simeq K.(PWV)^\\beta$")
ax2.axhline(0,c='k',ls="-.")
ax2.axvspan(0.15, 0.2, alpha=0.5, color='grey')
plt.tight_layout()
plt.show()

In [ ]:
def PlotTheFit(the_selected_good_index):
    """
    """
    
    
  
    yylabel = "$|\delta_{12}C_{zy}|\; (mmag)$"
    xxlabel = "$PWV$ (mm) in direction obs"
    fitter = FitAtmosphericColorParams()

    df_fit_ap = pd.DataFrame(columns=["idx_pair","idx_summary","color","slope","slope_err","sep"]) 
    date_form = DateFormatter("%y-%m-%d")

    count = 0
    #for good_idx in good_indexes:
    for idx,good_idx in enumerate(good_indexes):
        
        if good_idx != the_selected_good_index:
            continue
        
        
        # get the color now
        df_col = all_colors_diff[good_idx]  
        #Extract the row in df_summary with the pair number good_idx
        df_summary_row = df_summary[df_summary["idx_pair"].astype(int) == good_idx]
        
        
        print("===================================================================================")
        print(df_summary_row )
        #print(df_col) Too long
        
        # extract the corresponding index and refcol_ap_md column
        index_in_summary = df_summary_row.index[0]
        refcol_ap_md = df_summary_row["refcol_ap_md"].values[0]
        print(f"\t ==> refcol_ap_md = {refcol_ap_md:.3f} mag")
        
    
        
        # compute a recentered colors fro plotting
        df_col["relapcol"] = df_col.apcol12_s12-refcol_ap_md

        separation_angle = df_summary.loc[index_in_summary,"sep"]
        
        
        # get the light curve
        PWV = df_col["PWV [mm]_x"].values 
        X =  df_col["Time"].values
        Y =  (df_col["PWV [mm]_x"] *  df_col["airmassb1"]).values
        Yerr = (df_col["PWV [mm]_err_x"]*df_col["airmassb1"]).values
        DT = df_col["dt_abs_specphot"].values # time difference
        CHI2 = df_col["CHI2_FIT"].values
        

        goods = (~np.isnat(X)) & (DT < DTMAX)  & (CHI2<CHI2FITMAX) & (PWV>0.1) & (PWV<9.9)
        
        X=X[goods]
        Y=Y[goods]
        Yerr = Yerr[goods]
        DT = DT[goods]
        
      
        
        # the color diff light curve where all usefull variable are selected
        df_col_subset = df_col[["Time","PWV [mm]_x","PWV [mm]_err_x","dt_abs_specphot","airmassb1","relapcol","apcol12Err_s12","CHI2_FIT"]][goods]
        df_col_subset["pwvxairmass"] = df_col_subset["PWV [mm]_x"]*df_col_subset["airmassb1"]
        
        
        tmin = df_col_subset["Time"].min()
        tmax = df_col_subset["Time"].max()
        dt = tmax-tmin
        tmin = tmin-dt/10
        tmax = tmax+dt/10
        the_tmin=tmin
        the_tmax=tmax
        
        #DT bin colors
        jet = plt.get_cmap('jet')
        cNorm = mpl.colors.Normalize(vmin=DT.min(), vmax=DT.max())
        scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)
        all_colors_dt = scalarMap.to_rgba(DT, alpha=1)
        
        if len(X)<5:
            print(f"{idx}) pair {good_idx} rejected too small number of coincidence photom/spectro ")
            continue
        
   
        
        # the SED model not used anymore    
        the_color_curve = all_color_curves[idx]
        the_color_curve1 = all_color_curves1[idx]
        the_color_curve2 = all_color_curves2[idx]

        # decomposition of the figure in 4 subplots
        # =========================================
        fig = plt.figure(figsize=(12,8),constrained_layout=True)
        ax = fig.add_subplot(111)

    


        # compute the string
        textstr0 = '\n'.join((
        "(Z-Y) diff star pair :"
        r'objects =(%.0f,%.0f)' % (df_summary.loc[index_in_summary,"idx_obj1"],df_summary.loc[index_in_summary,"idx_obj2"],  ),
        r'$\theta_{sep} = %.0f$ arcsec' % (df_summary.loc[index_in_summary,"sep"], ),
        r'$|\Delta_{12}C_{zy}|_0 \simeq%.3f$  mag' % ( refcol_ap_md, )))
        
        
        ax.text(0.6, 0.99, textstr0, transform=ax.transAxes, fontsize=14,verticalalignment='top', bbox=props)

        title = "pair {} :: obj1= {} obj2 = {} ".format(good_idx,df_summary.loc[index_in_summary,"idx_obj1"].astype(int),
                                                    df_summary.loc[index_in_summary,"idx_obj2"].astype(int))
        
        ax.set_title(title,fontsize=16)
        
        
      
        #sSelection of the points
        # ============================================
        
        
        X =  df_col_subset["pwvxairmass"].values
        Y =  df_col_subset["relapcol"].values*1000.
        Xerr = (df_col_subset["PWV [mm]_err_x"]*df_col_subset["airmassb1"]).values
        Yerr = df_col_subset.apcol12Err_s12.values*1000.
        
        
        # remove nan in X
        isgood = ~np.isnan(X)
        X = X[isgood]
        Y = Y[isgood]
             
        Xerr = Xerr[isgood]
        Yerr = Yerr[isgood]
        
      
        # Fit
        #===================
        # ax4.set_axis_off()
    
        params0 = [0.,0.]
    
        res_fit,fit_dict = fitter.fit_linearcolorterm(params0,X,Y,Yerr)
                                     
        cs = res_fit.x[0]
        cslope = res_fit.x[1]
        cslope_err = fit_dict["sigmas"][1]
        
        
        chi2ndeg = fit_dict["chi2_per_deg"]
        
        ax.errorbar(X,Y+cs,xerr=Xerr,yerr=Yerr,ecolor="k",fmt=".",label="data")
        ax.scatter(X,Y+cs,c="b",marker= "o",s=50)
        df_col_subset["refcolshift_3"] = Y+cs
        df_col_subset.plot.scatter(x="pwvxairmass",y="refcolshift_3",ax=ax,marker="o",c="dt_abs_specphot",s=50,colormap="jet")
        
        xfit = np.linspace(0.,15.,50)
        yfit = cslope*xfit
        
        ax.plot(xfit,yfit,"r-",label="linear color model")
        ax.legend(loc="upper left") 
        ax.grid()
        
        textstr1 = '\n'.join(("Linear fit : ","$\chi^2/ndf = $" + f"{chi2ndeg:.1f}",f"slope = {cslope:.3f} mmag/mm"))
        ax.text(0.7, 0.25, textstr1, transform=ax.transAxes, fontsize=14,color="r",verticalalignment='top', bbox=props)
      
        
        ax.set_xlabel(xxlabel)
        ax.set_ylabel(yylabel)
     
        
    
    
    
        params0 = [0.,1.]
    
        res_fit,fit_dict = fitter.fit_powerlawcolorterm(params0,X,Y,Yerr)
                                     
        cs = res_fit.x[0]
        K = res_fit.x[1]
        K_err =  fit_dict["sigmas"][1]
        
        chi2ndeg = fit_dict["chi2_per_deg"]
        
        #ax8.errorbar(X,Y+cs,xerr=Xerr,yerr=Yerr,ecolor="k",fmt=".",label="data")
        #ax8.scatter(X,Y+cs,c="b",marker= "o",s=50)
        #df_col_subset["refcolshift_4"] = Y+cs
        #df_col_subset.plot.scatter(x="pwvxairmass",y="refcolshift_4",ax=ax8,marker="o",c="dt_abs_specphot",s=50,colormap="jet")
        
        xfit = np.linspace(0.,15.,50)
        yfit = K*np.power(xfit,0.773)
        
        ax.plot(xfit,yfit,"b-",label="fit : $K.PWV^\\beta$")
        ax.legend(loc="upper left") 
        ax.grid()
        
        textstr2 = '\n'.join(("Fit $K.PWV^\\beta$ :","$\chi^2/ndf = $" + f"{chi2ndeg:.1f}",f"K = {K:.3f} mmag"))
        ax.text(0.7, 0.12, textstr2, transform=ax.transAxes, fontsize=14,color="b",verticalalignment='top', bbox=props)
        
       
        ax.axhline(0,c="k",ls="-.")
        
        cbar = ax.collections[-1].colorbar
        cbar.set_label("$\Delta t$ (s)")
        
        
        count += 1
        #plt.tight_layout()
        plt.show()
        
        
        
        #if count > 2:
        #    break
   


In [ ]:
PlotTheFit(93)

In [ ]:
PlotTheFit(505)

In [ ]:
ax.collections[-1].colorbar